<a href="https://colab.research.google.com/github/Ajinkya-18/NeuroVision/blob/main/eeg_text_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
import os
os.makedirs('/content/final_lightweight_17k', exist_ok=True)

In [2]:
!cp /content/drive/MyDrive/NeuroVision/alljoined_lightweight_17k.tar.gz /content/

In [3]:
!tar -xzf /content/alljoined_lightweight_17k.tar.gz -C /content/final_lightweight_17k

In [4]:
import torch
from pathlib import Path
from tqdm.auto import tqdm

# Path to your lightweight dataset on the local Colab disk
DATA_ROOT = '/content/final_lightweight_17k'
SPECTROGRAM_TRAIN_DIR = Path(DATA_ROOT) / 'spectrograms' / 'train'

# Initialize accumulators
channel_sum = torch.zeros(64, dtype=torch.float64)
channel_sum_sq = torch.zeros(64, dtype=torch.float64)
pixel_count = 0

# Calculate stats only on the training set
files = list(SPECTROGRAM_TRAIN_DIR.glob('*.pt'))
for path in tqdm(files, desc="Calculating Spectrogram Stats"):
    data = torch.load(path)
    channel_sum += data.sum(dim=[1, 2]).to(torch.float64)
    channel_sum_sq += (data.to(torch.float64) ** 2).sum(dim=[1, 2])
    pixel_count += data.shape[1] * data.shape[2]

mean = (channel_sum / pixel_count).to(torch.float32)
std = torch.sqrt((channel_sum_sq / pixel_count) - mean.to(torch.float64)**2).to(torch.float32)

# Save the stats to the dataset folder
torch.save(mean, Path(DATA_ROOT) / 'spec_mean.pt')
torch.save(std, Path(DATA_ROOT) / 'spec_std.pt')

print(f"\nStats calculated and saved to {DATA_ROOT}")

Calculating Spectrogram Stats:   0%|          | 0/14000 [00:00<?, ?it/s]


Stats calculated and saved to /content/final_lightweight_17k


In [5]:
print("⏳ Installing and upgrading libraries...")
# The -q flag makes the output cleaner
!pip install torch torchvision timm lpips transformers accelerate diffusers open-clip-torch
!pip install peft ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

⏳ Installing and upgrading libraries...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.6 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-nmp3yvwe
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-nmp3yvwe
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=f0f7f380d5a28f4513bffb4884e6158611a7b34e03552c7a8c1006f2d03c0005
  Stored in directory: /tmp/pip-ephem-wheel-cache-suvknqvr/wheels/35/3e/df/3d24cbfb3b6a06f17a2bfd7d1138900d4365d9028aa8f6e92f
Successfully built clip


In [6]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Training

In [ ]:
import torch
import pandas as pd
from pathlib import Path
from PIL import Image
from tqdm.auto import tqdm
import json
from transformers import Blip2Processor, Blip2ForConditionalGeneration

# --- CONFIGURATION ---
class CONFIG:
    PROCESSED_DATA_ROOT = '/content/final_lightweight_17k'
    METADATA_CSV = Path(PROCESSED_DATA_ROOT) / 'metadata.csv'
    BLIP_MODEL_NAME = "Salesforce/blip2-opt-2.7b"

def run_caption_generation():
    """A self-contained script to run only once."""
    config = CONFIG()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    print("Loading BLIP model for caption generation...")
    processor = Blip2Processor.from_pretrained(config.BLIP_MODEL_NAME)
    caption_blip = Blip2ForConditionalGeneration.from_pretrained(config.BLIP_MODEL_NAME).to(device)

    for split in ['train', 'val']:
        captions_path = Path(config.PROCESSED_DATA_ROOT) / f'{split}_gt_captions.json'
        if captions_path.exists():
            print(f"Captions for '{split}' split already exist. Skipping.")
            continue

        print(f"Generating captions for '{split}' split...")
        df = pd.read_csv(config.METADATA_CSV)
        split_df = df[df['split'].str.strip() == split].reset_index(drop=True)

        captions = {}
        batch_size = 16
        caption_blip.eval()

        with torch.no_grad():
            for i in tqdm(range(0, len(split_df), batch_size), desc=f"Generating {split} captions"):
                batch_info = split_df.iloc[i:i+batch_size]
                batch_images = [
                    Image.open(Path(config.PROCESSED_DATA_ROOT) / info['image_path']).convert("RGB")
                    for _, info in batch_info.iterrows()
                ]

                inputs = processor(images=batch_images, return_tensors="pt").to(device)
                generated_ids = caption_blip.generate(**inputs, max_length=50)
                batch_captions = processor.batch_decode(generated_ids, skip_special_tokens=True)

                for j, caption in enumerate(batch_captions):
                    captions[str(i + j)] = caption.strip()

        with open(captions_path, 'w') as f:
            json.dump(captions, f, indent=2)
        print(f"✅ Saved {len(captions)} captions for '{split}' split.")

if __name__ == '__main__':
    run_caption_generation()

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Loading BLIP model for caption generation...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

...

Generating captions for 'train' split...


Generating train captions:   0%|          | 0/875 [00:00<?, ?it/s]

✅ Saved 14000 captions for 'train' split.
Generating captions for 'val' split...


Generating val captions:   0%|          | 0/175 [00:00<?, ?it/s]

✅ Saved 2800 captions for 'val' split.


In [ ]:
!apt-get install -y fonts-liberation

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-liberation is already the newest version (1:1.07.4-11).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [ ]:
import torch
import torch.nn as nn
import pandas as pd
from pathlib import Path
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torchvision.transforms import v2 as transforms
from PIL import Image
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.auto import tqdm
from datetime import datetime
import numpy as np
from transformers import Blip2Processor, Blip2ForConditionalGeneration, Blip2VisionModel
import warnings
from sentence_transformers import SentenceTransformer
from diffusers import StableDiffusionPipeline
import torch.nn.functional as F
from PIL import Image, ImageDraw, ImageFont
import json
from transformers import get_linear_schedule_with_warmup

# ==============================================================================
# --- 1. CONFIGURATION ---
# ==============================================================================
class TRAIN_CONFIG:
    PROCESSED_DATA_ROOT = '/content/final_lightweight_17k'
    METADATA_CSV = Path(PROCESSED_DATA_ROOT) / 'metadata.csv'
    OUTPUT_DIR = '/content/drive/MyDrive/NeuroVision/EEG_BLIP2_Alignment'

    BLIP_MODEL_NAME = "Salesforce/blip2-opt-2.7b"

    BATCH_SIZE = 14
    GRAD_ACCUMULATION_STEPS = 2
    NUM_EPOCHS = 30
    ADAPTER_LR = 1e-5  # Higher LR for the new adapter
    BLIP_LR = 1e-6     # Lower LR for fine-tuning the pre-trained layers

    WEIGHT_DECAY = 1e-3
    GRADIENT_CLIP_NORM = 1.0

    VALIDATION_INTERVAL = 1
    VAL_SAMPLES_PER_EPOCH = 400
    TRAIN_SAMPLES_PER_EPOCH = 2000
    VIS_GRID_SIZE = 5

    DIFFUSION_MODEL_ID ="runwayml/stable-diffusion-v1-5"
    EVAL_IMAGE_GENERATION_INTERVAL = 1
    WARMUP_STEPS = 500  # Number of steps to gradually increase the LR
    TOTAL_TRAIN_STEPS = (TRAIN_SAMPLES_PER_EPOCH/BATCH_SIZE) * NUM_EPOCHS
    STAGE1_EPOCHS = 12
    # STAGE2_EPOCHS = 18
    ALIGN_WEIGHT = 1.0
    SEMANTIC_WEIGHT = 2.0
    CE_WEIGHT = 3.0
    BEST_MODEL_DIR = '/content/drive/MyDrive/NeuroVision/EEG_BLIP2_Alignment/run_20251001_103040/best_model_epoch_27_loss_5.0528.pth'


# ==============================================================================
# --- 2. EEG ENCODER MODULE ---
# ==============================================================================

class EEGTransformerEncoder(nn.Module):
    """
    A powerful hybrid CNN-Transformer encoder to extract rich features
    directly from 64-channel EEG spectrograms.
    """
    def __init__(self, in_chans=64, embed_dim=768, nhead=8, num_layers=4, dropout=0.3):
        super().__init__()

        # 1. Convolutional Stem: Extracts local spatio-temporal features
        self.conv_stem = nn.Sequential(
            nn.Conv2d(in_chans, embed_dim // 4, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(embed_dim // 4),
            nn.ELU(),
            nn.Conv2d(embed_dim // 4, embed_dim // 2, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(embed_dim // 2),
            nn.ELU(),
            nn.Conv2d(embed_dim // 2, embed_dim, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(embed_dim),
        )

        # 2. Transformer Encoder: Learns global relationships between features
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=nhead,
            dim_feedforward=embed_dim * 4,
            dropout=dropout,
            activation='gelu',
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.output_layernorm = nn.LayerNorm(embed_dim)

    def forward(self, x):
        # Input x: [Batch, 64, Freq, Time]

        # Pass through convolutional stem
        x = self.conv_stem(x)
        # Output x: [Batch, embed_dim, F/4, T/4]

        # Prepare for transformer: flatten spatial dims and permute
        b, c, h, w = x.shape
        x = x.flatten(2).permute(0, 2, 1) # -> [Batch, H*W, embed_dim]

        # Pass through transformer encoder
        x = self.transformer_encoder(x)

        # Apply final layer normalization
        x = self.output_layernorm(x)

        return x

# ==============================================================================
# --- 3. THE HYBRID EEG-BLIP2 MODEL ---
# ==============================================================================
class EEG_BLIP2_Model(nn.Module):
    """
    The final, refactored model that directly connects a powerful EEG encoder
    to the BLIP-2 Q-Former via a projection layer.
    """
    def __init__(self, config, device, eeg_embed_dim=768):
        super().__init__()
        self.device = device

        # 1. Instantiate the powerful, from-scratch EEG encoder
        self.eeg_encoder = EEGTransformerEncoder(embed_dim=eeg_embed_dim)

        # 2. Load the pre-trained BLIP model
        self.blip = Blip2ForConditionalGeneration.from_pretrained(config.BLIP_MODEL_NAME)

        # 3. Create the crucial projection layer
        # This layer maps the EEG encoder's output dimension (e.g., 768) to the
        # dimension the BLIP vision encoder's features, which the Q-Former expects (1408).
        blip_vision_hidden_size = self.blip.vision_model.config.hidden_size
        self.eeg_projection = nn.Linear(eeg_embed_dim, blip_vision_hidden_size)

        # The __init__ method no longer handles freezing. This is now controlled by the training loop.

        self.eeg_encoder.to(device)
        self.eeg_projection.to(device)
        self.blip.to(device)

    def get_eeg_embedding(self, eeg_spectrograms):
        """ The direct EEG-to-Q-Former forward pass. """
        # 1. Get rich features from our custom EEG encoder
        eeg_features = self.eeg_encoder(eeg_spectrograms)
        # Output shape: [Batch, SeqLen, 768]

        # 2. Project EEG features to match the Q-Former's expected input dimension
        projected_eeg_features = self.eeg_projection(eeg_features)
        # Output shape: [Batch, SeqLen, 1408]

        # 3. Pass projected features directly to the Q-Former
        query_tokens = self.blip.query_tokens.expand(projected_eeg_features.shape[0], -1, -1)
        query_outputs = self.blip.qformer(
            query_embeds=query_tokens,
            encoder_hidden_states=projected_eeg_features,
            return_dict=True,
        )
        return query_outputs.last_hidden_state.mean(dim=1)

    def get_image_embedding(self, pil_images, processor):
        # This function remains the same, using the standard image pathway
        inputs = processor(images=pil_images, return_tensors="pt").to(self.device)
        pixel_values = inputs.pixel_values
        image_features = self.blip.vision_model(pixel_values).last_hidden_state
        query_tokens = self.blip.query_tokens.expand(image_features.shape[0], -1, -1)
        query_outputs = self.blip.qformer(
            query_embeds=query_tokens,
            encoder_hidden_states=image_features,
            return_dict=True,
        )
        return query_outputs.last_hidden_state.mean(dim=1)

    def generate(self, eeg_spectrograms, **kwargs):
        with torch.no_grad():
            # Follow the same direct pathway for generation
            eeg_features = self.eeg_encoder(eeg_spectrograms)
            projected_eeg_features = self.eeg_projection(eeg_features)

            query_tokens = self.blip.query_tokens.expand(projected_eeg_features.shape[0], -1, -1)
            query_outputs = self.blip.qformer(
                query_embeds=query_tokens,
                encoder_hidden_states=projected_eeg_features,
                return_dict=True,
            )
            language_model_inputs = self.blip.language_projection(query_outputs.last_hidden_state)
            generated_ids = self.blip.language_model.generate(
                inputs_embeds=language_model_inputs,
                **kwargs
            )
            return generated_ids

    def forward(self, eeg_spectrograms):
        """
        Returns the EEG embedding and the raw logits from the language model.
        """
        # 1. Get EEG features and project them
        eeg_features = self.eeg_encoder(eeg_spectrograms)
        projected_eeg_features = self.eeg_projection(eeg_features)

        # 2. Get the 32 query embeddings from the Q-Former
        query_tokens = self.blip.query_tokens.expand(projected_eeg_features.shape[0], -1, -1)
        query_outputs = self.blip.qformer(
            query_embeds=query_tokens,
            encoder_hidden_states=projected_eeg_features,
            return_dict=True,
        )

        # 3. Project queries for the language model
        language_model_inputs = self.blip.language_projection(query_outputs.last_hidden_state)

        # 4. Get the logits from the language model
        outputs = self.blip.language_model(inputs_embeds=language_model_inputs, return_dict=True)
        logits = outputs.logits

        # Get the final EEG embedding for our alignment loss
        eeg_embedding = query_outputs.last_hidden_state.mean(dim=1)

        return eeg_embedding, logits

# ==============================================================================
# --- 4. DATASET & COLLATING ---
# ==============================================================================
def collate_fn(batch):
    spectrograms, pil_images, text_captions = zip(*[(item[0], item[1], item[2]) for item in batch])
    labels = [item[3] for item in batch]
    padded_labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=-100)

    return torch.stack(spectrograms), list(pil_images), list(text_captions), padded_labels


class EEGDatasetWithCaptions(Dataset):
    def __init__(self, root_dir, metadata_csv, split, processor, transform, augment=False):
        self.root_dir = Path(root_dir)
        self.transform = transform
        df = pd.read_csv(metadata_csv)
        self.split_df = df[df['split'].str.strip() == split].reset_index(drop=True)
        self.augment = augment

        self.processor = processor

        # --- NEW: Define a dedicated augmentation pipeline ---
        if self.augment:
            self.augmentation_transform = transforms.Compose([
                transforms.RandomApply([transforms.GaussianBlur(kernel_size=3)], p=0.5),
                # Randomly mask out frequency bands
                transforms.RandomApply([transforms.RandomErasing(p=1.0, scale=(0.02, 0.1), ratio=(0.1, 0.5))], p=0.5),
                # Randomly mask out time steps
                transforms.RandomApply([transforms.RandomErasing(p=1.0, scale=(0.02, 0.1), ratio=(2.0, 5.0))], p=0.5),
            ])

        # This is your existing normalization transform
        self.normalization_transform = transform

        # Load pre-generated captions
        captions_path = Path(root_dir) / f'{split}_gt_captions.json'
        print(f"Loading pre-generated captions from {captions_path}")
        with open(captions_path, 'r') as f:
            self.captions = json.load(f)

    def __len__(self):
        return len(self.split_df)

    def __getitem__(self, idx):
        info = self.split_df.iloc[idx]

        spectrogram = torch.load(self.root_dir / info['spectrogram_path'])

        # --- APPLY AUGMENTATIONS (for training set only) ---
        if self.augment:
            spectrogram = self.augmentation_transform(spectrogram)

        # Apply normalization after augmentation
        spectrogram = self.normalization_transform(spectrogram)

        image = Image.open(self.root_dir / info['image_path']).convert("RGB")
        gt_caption = self.captions.get(str(idx), "an image")
        labels = self.processor(text=gt_caption, return_tensors="pt", padding=True).input_ids.squeeze()

        return spectrogram, image, gt_caption, labels

# ==============================================================================
# --- 5. MAIN TRAINING FUNCTION ---
# ==============================================================================
# InfoNCE loss function

def info_nce_loss(query, positive_key, temperature=0.07):
    # Ensure inputs are normalized
    query = F.normalize(query, dim=-1)
    positive_key = F.normalize(positive_key, dim=-1)

    # Calculate the similarity matrix of every query with every key
    # The diagonal of this matrix contains the positive pairs
    logits = query @ positive_key.T

    # The labels are the indices of the positive pairs (the diagonal)
    labels = torch.arange(len(query), device=query.device)

    # Calculate the cross-entropy loss
    return F.cross_entropy(logits / temperature, labels)


class SemanticLoss(nn.Module):
    """
    A loss function that measures the semantic similarity between two sets of captions
    using a pre-trained SentenceTransformer model.
    """
    def __init__(self, device):
        super().__init__()
        self.device = device
        # Load a pre-trained model optimized for semantic similarity
        self.model = SentenceTransformer('all-MiniLM-L6-v2').to(device)
        # Freeze the model's weights as we only use it for inference
        for param in self.model.parameters():
            param.requires_grad = False

    def forward(self, predicted_captions, ground_truth_captions):
        # Convert the text captions into semantic embedding vectors
        pred_embeddings = self.model.encode(predicted_captions, convert_to_tensor=True)
        gt_embeddings = self.model.encode(ground_truth_captions, convert_to_tensor=True)

        # Calculate the cosine similarity. The loss is 1.0 minus the similarity.
        # A higher similarity (closer to 1.0) results in a lower loss (closer to 0.0).
        cosine_sim = F.cosine_similarity(pred_embeddings, gt_embeddings, dim=-1)
        loss = 1.0 - cosine_sim.mean()
        return loss

#------------------------------------------------------------------------------------------

def create_and_save_reconstruction_grid(eval_samples, diffusion_pipe, epoch, val_loss, output_dir):
    """
    Generates a 2x2 image grid comparing original and reconstructed images.
    Top row: Original images with ground truth captions.
    Bottom row: Reconstructed images with predicted captions.
    """
    if len(eval_samples) < 2:
        print("Not enough samples for a 2x2 grid, skipping.")
        return

    print(f"\n🖼️  Generating reconstruction grid for epoch {epoch}...")

    # Setup canvas and font
    w, h = 512, 512  # Standard Stable Diffusion size
    title_h = 60    # Space for titles
    grid = Image.new('RGB', (w * 2, h * 2 + title_h * 2), 'black')
    draw = ImageDraw.Draw(grid)
    try:
        font = ImageFont.truetype("LiberationSans-Regular.ttf", 20)
    except IOError:
        print("Default font not found, using fallback.")
        font = ImageFont.load_default()

    # Process two samples
    for i in range(2):
        predicted_cap, gt_cap, original_image = eval_samples[i]

        # --- Generate the reconstructed image from the predicted caption ---
        generator = torch.Generator(device=diffusion_pipe.device).manual_seed(42 + i)
        reconstructed_image = diffusion_pipe(
            prompt=predicted_cap, generator=generator, num_inference_steps=20
        ).images[0]

        # --- Paste images and draw titles ---
        col_offset = i * w

        # Top Row (Original)
        draw.text((col_offset + 5, 5), f"GT: {gt_cap[:50]}...", font=font, fill="white")
        grid.paste(original_image.resize((w, h)), (col_offset, title_h))

        # Bottom Row (Reconstructed)
        draw.text((col_offset + 5, h + title_h + 5), f"Pred: {predicted_cap[:50]}...", font=font, fill="white")
        grid.paste(reconstructed_image.resize((w, h)), (col_offset, h + title_h * 2))

    # --- Save the final grid ---
    # Sanitize loss value for filename
    val_loss_str = f"{val_loss:.4f}".replace('.', '_')
    save_path = Path(output_dir) / 'reconstructions'
    save_path.mkdir(exist_ok=True)
    filename = save_path / f"reconstruction_epoch_{epoch}_loss_{val_loss_str}.png"
    grid.save(filename)
    print(f"  Saved reconstruction grid to {filename}")

#----------------------------------------------------------------------------------------------
def train_model(config):
    # --- Setup (Device, Output Dir, etc.) ---
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    output_dir = Path(config.OUTPUT_DIR) / f'run_{timestamp}'
    output_dir.mkdir(parents=True, exist_ok=True)

    # --- Models and Data ---
    processor = Blip2Processor.from_pretrained(config.BLIP_MODEL_NAME)
    model = EEG_BLIP2_Model(config, device) # Assumes the refactored, cleaner version

    for p in model.blip.parameters():
        p.requires_grad=False

    print("Starting training from scratch.")
    # try:
    #   print('Loading best model weights')
    #   model.load_state_dict(torch.load(config.BEST_MODEL_DIR, map_location=device))
    #   print('Successfully loaded best model weights!')
    # except:
    #   print('Could not load the best model weights.')

    print("Loading Stable Diffusion pipeline for visualizations...")
    diffusion_pipe = StableDiffusionPipeline.from_pretrained(
        config.DIFFUSION_MODEL_ID,
        torch_dtype=torch.float16 if device.type == "cuda" else torch.float32,
        safety_checker=None, requires_safety_checker=False
    ).to(device)
    diffusion_pipe.set_progress_bar_config(disable=True)

    semantic_loss_fn = SemanticLoss(device)
    ce_loss_fn = nn.CrossEntropyLoss(ignore_index=-100)

    # --- Data Loading ---
    spec_mean = torch.load(Path(config.PROCESSED_DATA_ROOT) / 'spec_mean.pt')
    spec_std = torch.load(Path(config.PROCESSED_DATA_ROOT) / 'spec_std.pt')
    spec_std += 1e-6

    transform = transforms.Compose([
        transforms.ToDtype(torch.float32),
        transforms.Resize((224, 224), antialias=True),
        transforms.Normalize(mean=spec_mean.tolist(), std=spec_std.tolist()),
    ])

    train_dataset = EEGDatasetWithCaptions(config.PROCESSED_DATA_ROOT, config.METADATA_CSV, 'train', processor, transform, augment=True)
    val_dataset = EEGDatasetWithCaptions(config.PROCESSED_DATA_ROOT, config.METADATA_CSV, 'val', processor, transform, augment=False)

    # --- Training Components ---
    scaler = torch.cuda.amp.GradScaler()
    best_val_loss = float('inf')

    generation_args = {
        "max_new_tokens": 15, "do_sample": True, "min_new_tokens": 5,
        "repetition_penalty": 1.4, "top_p": 0.9, "temperature": 0.85
    }

    # --- Main Training Loop ---
    print("🎯 Starting Decoupled 3-Stage Training Loop")
    for epoch in range(config.NUM_EPOCHS):

        # --- STAGE 1 & 2 Freezing Logic ---
        if epoch < config.STAGE1_EPOCHS:
            if epoch == 0:
                print(f"\n--- STAGE 1: Training EEG Encoder & Projection ONLY (Epochs 0-{config.STAGE1_EPOCHS-1}) ---")

            # Set requires_grad for Stage 1
            for name, param in model.named_parameters():
                if 'eeg_encoder' in name or 'eeg_projection' in name:
                    param.requires_grad = True
                    # print('Unfreeing Encoder and eeg-projection layers')

                else:
                    param.requires_grad = False

        elif epoch == config.STAGE1_EPOCHS:
            print(f"\n--- STAGE 2: Fine-Tuning Q-Former and language-projection (Epochs {config.STAGE1_EPOCHS}+) ---")

            # Set requires_grad for Stage 2
            for name, param in model.named_parameters():
                if 'qformer' in name or 'language_projection' in name:
                    param.requires_grad = True
                    # print('Unfreezing Q-Former layer')

                else:
                    param.requires_grad = False

        # elif epoch == config.STAGE2_EPOCHS:
        #     print(f"\n---STAGE 3: Fine-tuning Language Projection (Epochs {config.STAGE2_EPOCHS}+)---")

        #     for name, param in model.named_parameters():
        #         if 'qformer' in name or 'language_projection' in name:
        #             param.requires_grad = True
        #             # print('Unfreezing language-projection layer')

        #         else:
        #             param.requires_grad = False


        # --- Create/Re-create Optimizer & Scheduler at stage transitions ---
        if epoch == 0 or epoch == config.STAGE1_EPOCHS:
            print(f"Creating optimizer for new stage...")

            # Use differential learning rates for Stage 2 and 3
            if epoch < config.STAGE1_EPOCHS:
                trainable_params = filter(lambda p: p.requires_grad, model.parameters())
                optimizer = optim.AdamW(trainable_params, lr=config.ADAPTER_LR, weight_decay=config.WEIGHT_DECAY)
            else: # For both Stage 2 and Stage 3
                qformer_params = [p for n, p in model.named_parameters() if p.requires_grad and 'qformer' in n]
                lang_proj_params = [p for n, p in model.named_parameters() if p.requires_grad and 'language_projection' in n]

                optimizer_grouped_parameters = [
                    {"params": qformer_params, "lr": config.BLIP_LR},
                    {"params": lang_proj_params, "lr": config.BLIP_LR * 0.5}, # Even smaller LR for the final layer
                ]
                optimizer = optim.AdamW(optimizer_grouped_parameters, weight_decay=config.WEIGHT_DECAY)

            num_training_steps = (len(train_dataset) // config.BATCH_SIZE) * config.NUM_EPOCHS
            scheduler = get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=config.WARMUP_STEPS, num_training_steps=num_training_steps
            )
            print("Optimizer and scheduler created.")

        # --- Training Batch Loop ---
        model.train()
        train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE,
            sampler=SubsetRandomSampler(np.random.choice(len(train_dataset), min(len(train_dataset), config.TRAIN_SAMPLES_PER_EPOCH), replace=False)),
            drop_last=True, collate_fn=collate_fn)
        train_bar = tqdm(train_loader, desc=f"🔄 Epoch {epoch+1}/{config.NUM_EPOCHS}")

        for batch_idx, (spectrograms, pil_images, gt_text_captions, labels) in enumerate(train_bar):
            spectrograms, labels = spectrograms.to(device), labels.to(device)

            with torch.amp.autocast('cuda'):
                eeg_embeddings, logits = model(spectrograms)

                # For semantic loss, we need to generate captions (this is slower but necessary)
                with torch.no_grad():
                    image_embeddings = model.get_image_embedding(pil_images, processor)
                    generated_ids = model.generate(spectrograms, **generation_args)
                    eeg_captions = processor.batch_decode(generated_ids, skip_special_tokens=True)

                loss_semantic = semantic_loss_fn(eeg_captions, gt_text_captions)
                loss_align = info_nce_loss(eeg_embeddings, image_embeddings)

                logits_len = logits.size(1)
                labels_len = labels.size(1)

                if labels_len < logits_len:
                    padded_labels = F.pad(labels, (0, logits_len - labels_len), 'constant', -100)
                else:
                    # In case labels are longer (unlikely), truncate them
                    padded_labels = labels[:, :logits_len]

                loss_ce = ce_loss_fn(logits.view(-1, logits.size(-1)), padded_labels.view(-1))

                # Use consistent loss weights defined in config
                loss = (config.ALIGN_WEIGHT * loss_align) + (config.SEMANTIC_WEIGHT * loss_semantic) + (config.CE_WEIGHT * loss_ce)
                loss = loss / config.GRAD_ACCUMULATION_STEPS

            scaler.scale(loss).backward()

            if (batch_idx + 1) % config.GRAD_ACCUMULATION_STEPS == 0:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(filter(lambda p: p.requires_grad, model.parameters()), config.GRADIENT_CLIP_NORM)
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
                scheduler.step()

            train_bar.set_postfix({
                "ce_loss": f"{loss_ce.item():.4f}",
                "align_loss": f"{loss_align.item():.4f}",
                "sem_loss": f"{loss_semantic.item():.4f}"
            })

        # --- Validation Loop ---
        print(f"\n🔍 Running Validation for epoch {epoch+1}")
        model.eval()

        val_align_losses, val_semantic_losses, val_ce_losses, eval_samples = [], [], [], []

        val_loader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE,
            sampler=SubsetRandomSampler(np.random.choice(len(val_dataset), min(len(val_dataset), config.VAL_SAMPLES_PER_EPOCH), replace=False)),
            collate_fn=collate_fn)

        with torch.no_grad(), torch.amp.autocast('cuda'):
            for spectrograms, pil_images, gt_text_captions, labels in tqdm(val_loader, desc="Validation Loop"):
                spectrograms, labels = spectrograms.to(device), labels.to(device)

                eeg_embeddings, logits = model(spectrograms)
                image_embeddings = model.get_image_embedding(pil_images, processor)

                generated_ids = model.generate(spectrograms, **generation_args)
                eeg_captions = processor.batch_decode(generated_ids, skip_special_tokens=True)

                loss_align = info_nce_loss(eeg_embeddings, image_embeddings)
                loss_semantic = semantic_loss_fn(eeg_captions, gt_text_captions)

                logits_len = logits.size(1)
                labels_len = labels.size(1)
                if labels_len < logits_len:
                    padded_labels = F.pad(labels, (0, logits_len - labels_len), 'constant', -100)
                else:
                    padded_labels = labels[:, :logits_len]

                loss_ce = ce_loss_fn(logits.view(-1, logits.size(-1)), padded_labels.view(-1))

                val_align_losses.append(loss_align.item())
                val_semantic_losses.append(loss_semantic.item())
                val_ce_losses.append(loss_ce.item())

                if len(eval_samples) < config.VIS_GRID_SIZE:
                    for i in range(min(config.VIS_GRID_SIZE - len(eval_samples), len(eeg_captions))):
                        eval_samples.append((eeg_captions[i], gt_text_captions[i], pil_images[i]))

        avg_val_align_loss = np.mean(val_align_losses)
        avg_val_semantic_loss = np.mean(val_semantic_losses)
        avg_val_ce_loss = np.mean(val_ce_losses)

        total_val_loss = (config.ALIGN_WEIGHT * avg_val_align_loss) + \
                         (config.SEMANTIC_WEIGHT * avg_val_semantic_loss) + \
                         (config.CE_WEIGHT * avg_val_ce_loss)

        print(f"  Validation Results:")
        print(f"    - Avg Alignment Loss: {avg_val_align_loss:.4f}")
        print(f"    - Avg Semantic Loss:  {avg_val_semantic_loss:.4f}")
        print(f"    - Avg CE Loss:        {avg_val_ce_loss:.4f}")
        print(f"    - Total Weighted Loss: {total_val_loss:.4f}")

        if total_val_loss < best_val_loss:
            best_val_loss = total_val_loss
            save_path = output_dir / f'best_model_epoch_{epoch+1}_loss_{total_val_loss:.4f}.pth'
            torch.save(model.state_dict(), save_path)
            print(f"🏆 New best model saved to {save_path}")

        # --- Print caption comparisons ---
        if eval_samples:
            print("\n--- ✍️  Caption Generation Samples ---")
            for i, (predicted_cap, gt_cap, _) in enumerate(eval_samples):
                print(f"Sample {i+1}:")
                print(f"  - Ground Truth: {gt_cap.strip()}")
                print(f"  - Predicted:    {predicted_cap.strip()}")
            print("------------------------------------")

        if (epoch + 1) % config.EVAL_IMAGE_GENERATION_INTERVAL == 0 and eval_samples:
            create_and_save_reconstruction_grid(
                eval_samples,
                diffusion_pipe,
                epoch + 1,
                total_val_loss,
                output_dir
            )

    print("\n🎉 Training Complete!")
    return output_dir

if __name__ == '__main__':
    config = TRAIN_CONFIG()
    try:
        output_dir = train_model(config)
        if output_dir:
            print(f"✅ Training completed successfully! Results saved to: {output_dir}")
    except Exception as e:
        print(f"❌ Training failed with error: {str(e)}")
        import traceback
        traceback.print_exc()

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Starting training from scratch.
Loading Stable Diffusion pipeline for visualizations...


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

...

  Validation Results:
    - Avg Alignment Loss: 2.6434
    - Avg Semantic Loss:  0.9199
    - Avg CE Loss:        4.0519
    - Total Weighted Loss: 16.6389

--- ✍️  Caption Generation Samples ---
Sample 1:
  - Ground Truth: a vase with flowers in it
  - Predicted:    on a dark background, two women are walking in the park. One
Sample 2:
  - Ground Truth: a man riding a wave on a surfboard
  - Predicted:    a table covered with plates, a plate in the middle of it and
Sample 3:
  - Ground Truth: man holding a baseball bat
  - Predicted:    it's not the same girl.  it is a different blonde,
Sample 4:
  - Ground Truth: a man riding a wave on a surfboard
  - Predicted:    a tree with a man dressed as the pope and other people on it
Sample 5:
  - Ground Truth: a plate of food with meat and broccoli
  - Predicted:    A car is standing on a street in the background. The text says
------------------------------------

🖼️  Generating reconstruction grid for epoch 29...
  Saved reconstruction gri

🔄 Epoch 30/30:   0%|          | 0/142 [00:00<?, ?it/s]


🔍 Running Validation for epoch 30


Validation Loop:   0%|          | 0/29 [00:00<?, ?it/s]

  Validation Results:
    - Avg Alignment Loss: 2.6430
    - Avg Semantic Loss:  0.9190
    - Avg CE Loss:        4.1190
    - Total Weighted Loss: 16.8381

--- ✍️  Caption Generation Samples ---
Sample 1:
  - Ground Truth: a living room with a table and chairs
  - Predicted:    a forest of the trees
Sample 2:
  - Ground Truth: a couple of people sitting at a table outside of a building
  - Predicted:    a long time ago a man and his dog got into an argument.
Sample 3:
  - Ground Truth: a bed with a blanket and a lamp in a room
  - Predicted:    a man playing a guitar, sitting in front of an abstract image on
Sample 4:
  - Ground Truth: two people laying on snowboards
  - Predicted:    on your knees with a towel over them.
Sample 5:
  - Ground Truth: two men playing tennis on a court
  - Predicted:    a table with a chair, some plates and glasses is in the foreground
------------------------------------

🖼️  Generating reconstruction grid for epoch 30...
  Saved reconstruction grid to 

# Correct Scripts for documentation

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
from pathlib import Path
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torchvision.transforms import v2 as transforms
from PIL import Image
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.auto import tqdm
from datetime import datetime
import numpy as np
from transformers import Blip2Processor, Blip2ForConditionalGeneration, Blip2VisionModel
import warnings
from sentence_transformers import SentenceTransformer
from diffusers import StableDiffusionPipeline
import torch.nn.functional as F
from PIL import Image, ImageDraw, ImageFont
import json
from transformers import get_linear_schedule_with_warmup

# ==============================================================================
# --- 1. CONFIGURATION ---
# ==============================================================================
class TRAIN_CONFIG:
    PROCESSED_DATA_ROOT = '/content/final_lightweight_17k'
    METADATA_CSV = Path(PROCESSED_DATA_ROOT) / 'metadata.csv'
    OUTPUT_DIR = '/content/drive/MyDrive/NeuroVision/EEG_BLIP2_Alignment'

    BLIP_MODEL_NAME = "Salesforce/blip2-opt-2.7b"

    BATCH_SIZE = 8
    GRAD_ACCUMULATION_STEPS = 4
    NUM_EPOCHS = 50
    ADAPTER_LR = 1e-5  # Higher LR for the new adapter
    BLIP_LR = 1e-6     # Lower LR for fine-tuning the pre-trained layers

    WEIGHT_DECAY = 1e-3
    GRADIENT_CLIP_NORM = 2.0

    VALIDATION_INTERVAL = 1
    VAL_SAMPLES_PER_EPOCH = 400
    TRAIN_SAMPLES_PER_EPOCH = 2000
    VIS_GRID_SIZE = 5

    DIFFUSION_MODEL_ID ="runwayml/stable-diffusion-v1-5"
    EVAL_IMAGE_GENERATION_INTERVAL = 1
    WARMUP_STEPS = 400  # Number of steps to gradually increase the LR
    TOTAL_TRAIN_STEPS = 250 * NUM_EPOCHS
    STAGE1_EPOCHS = 15
    ALIGN_WEIGHT = 1.0
    SEMANTIC_WEIGHT = 2.5
    BEST_MODEL_DIR = '/content/drive/MyDrive/NeuroVision/EEG_BLIP2_Alignment/run_20251001_103040/best_model_epoch_27_loss_5.0528.pth'


# ==============================================================================
# --- 2. EEG ENCODER MODULE ---
# ==============================================================================

class DeeperEEGNetAdapter(nn.Module):
    """A deeper, more powerful EEGNet-inspired adapter."""
    def __init__(self, in_chans=64, out_chans=3, F1=16, D=2, F2=32, F3=64, dropout=0.5):
        super().__init__()
        # Block 1 (same as before)
        self.block1 = nn.Sequential(
            nn.Conv2d(1, F1, (1, 64), padding='same', bias=False),
            nn.BatchNorm2d(F1),
            nn.Conv2d(F1, F1 * D, (in_chans, 1), groups=F1, bias=False),
            nn.BatchNorm2d(F1 * D),
            nn.ELU(),
            nn.AvgPool2d((1, 4)),
            nn.Dropout(dropout)
        )
        # Block 2 (same as before)
        self.block2 = nn.Sequential(
            nn.Conv2d(F1 * D, F2, (1, 16), padding='same', groups=F1 * D, bias=False),
            nn.Conv2d(F2, F2, (1, 1), bias=False),
            nn.BatchNorm2d(F2),
            nn.ELU(),
            nn.AvgPool2d((1, 8)),
            nn.Dropout(dropout)
        )
        # --- NEW: Third block for more capacity ---
        self.block3 = nn.Sequential(
            nn.Conv2d(F2, F3, (1, 8), padding='same', groups=F2, bias=False),
            nn.Conv2d(F3, F3, (1, 1), bias=False),
            nn.BatchNorm2d(F3),
            nn.ELU(),
            nn.AvgPool2d((1, 2)),
            nn.Dropout(dropout)
        )
        self.final_projection = nn.Conv2d(F3, out_chans, (1, 1))
        self.adaptive_pool = nn.AdaptiveAvgPool2d((224, 224))

    def forward(self, x):
        x = x.view(x.size(0), 1, x.size(1), -1)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x) # Pass through the new block
        x = self.final_projection(x)
        x = self.adaptive_pool(x)
        return x

# ==============================================================================
# --- 3. THE HYBRID EEG-BLIP2 MODEL ---
# ==============================================================================
class EEG_BLIP2_Model(nn.Module):
    def __init__(self, config, device):
        super().__init__()
        self.device = device

        # --- REFACTOR: Instantiate the adapter directly ---
        self.adapter = DeeperEEGNetAdapter(in_chans=64)

        self.blip = Blip2ForConditionalGeneration.from_pretrained(config.BLIP_MODEL_NAME)

        # Freeze the entire BLIP model initially
        for param in self.blip.parameters():
            param.requires_grad = False

        self.adapter.to(device)
        self.blip.to(device)

    def get_eeg_embedding(self, eeg_spectrograms):
        # --- REFACTOR: Forward pass is now cleaner ---
        pseudo_image = self.adapter(eeg_spectrograms)
        vision_outputs = self.blip.vision_model(pseudo_image, return_dict=True)
        eeg_features = vision_outputs.last_hidden_state

        query_tokens = self.blip.query_tokens.expand(eeg_features.shape[0], -1, -1)
        query_outputs = self.blip.qformer(
            query_embeds=query_tokens,
            encoder_hidden_states=eeg_features,
            return_dict=True,
        )
        return query_outputs.last_hidden_state.mean(dim=1)

    def get_image_embedding(self, pil_images, processor):
        inputs = processor(images=pil_images, return_tensors="pt").to(self.device)
        pixel_values = inputs.pixel_values

        image_features = self.blip.vision_model(pixel_values).last_hidden_state
        query_tokens = self.blip.query_tokens.expand(image_features.shape[0], -1, -1)
        query_outputs = self.blip.qformer(
            query_embeds=query_tokens,
            encoder_hidden_states=image_features,
            return_dict=True,
        )
        return query_outputs.last_hidden_state.mean(dim=1)

    def generate(self, eeg_spectrograms, **kwargs):
        with torch.no_grad():
            # --- REFACTOR: Generation pass is also cleaner ---
            pseudo_image = self.adapter(eeg_spectrograms)
            vision_outputs = self.blip.vision_model(pseudo_image, return_dict=True)
            eeg_features = vision_outputs.last_hidden_state

            query_tokens = self.blip.query_tokens.expand(eeg_features.shape[0], -1, -1)
            query_outputs = self.blip.qformer(
                query_embeds=query_tokens,
                encoder_hidden_states=eeg_features,
                return_dict=True,
            )
            language_model_inputs = self.blip.language_projection(query_outputs.last_hidden_state)
            generated_ids = self.blip.language_model.generate(
                inputs_embeds=language_model_inputs,
                **kwargs
            )
            return generated_ids

# ==============================================================================
# --- 4. DATASET & COLLATING ---
# ==============================================================================
def collate_fn(batch):
    spectrograms = torch.stack([item[0] for item in batch])
    pil_images = [item[1] for item in batch]
    captions = [item[2] for item in batch]
    return spectrograms, pil_images, captions

class EEGDatasetWithCaptions(Dataset):
    def __init__(self, root_dir, metadata_csv, split, transform, augment=False):
        self.root_dir = Path(root_dir)
        self.transform = transform
        df = pd.read_csv(metadata_csv)
        self.split_df = df[df['split'].str.strip() == split].reset_index(drop=True)
        self.augment = augment

        # --- NEW: Define a dedicated augmentation pipeline ---
        if self.augment:
            self.augmentation_transform = transforms.Compose([
                transforms.RandomApply([transforms.GaussianBlur(kernel_size=3)], p=0.5),
                # Randomly mask out frequency bands
                transforms.RandomApply([transforms.RandomErasing(p=1.0, scale=(0.02, 0.1), ratio=(0.1, 0.5))], p=0.5),
                # Randomly mask out time steps
                transforms.RandomApply([transforms.RandomErasing(p=1.0, scale=(0.02, 0.1), ratio=(2.0, 5.0))], p=0.5),
            ])

        # This is your existing normalization transform
        self.normalization_transform = transform

        # Load pre-generated captions
        captions_path = Path(root_dir) / f'{split}_gt_captions.json'
        print(f"Loading pre-generated captions from {captions_path}")
        with open(captions_path, 'r') as f:
            self.captions = json.load(f)

    def __len__(self):
        return len(self.split_df)

    def __getitem__(self, idx):
        info = self.split_df.iloc[idx]

        spectrogram = torch.load(self.root_dir / info['spectrogram_path'])

        # --- APPLY AUGMENTATIONS (for training set only) ---
        if self.augment:
            spectrogram = self.augmentation_transform(spectrogram)

        # Apply normalization after augmentation
        spectrogram = self.normalization_transform(spectrogram)

        image = Image.open(self.root_dir / info['image_path']).convert("RGB")
        gt_caption = self.captions.get(str(idx), "an image")

        return spectrogram, image, gt_caption

# ==============================================================================
# --- 5. MAIN TRAINING FUNCTION ---
# ==============================================================================
# InfoNCE loss function

def info_nce_loss(query, positive_key, temperature=0.07):
    # Ensure inputs are normalized
    query = F.normalize(query, dim=-1)
    positive_key = F.normalize(positive_key, dim=-1)

    # Calculate the similarity matrix of every query with every key
    # The diagonal of this matrix contains the positive pairs
    logits = query @ positive_key.T

    # The labels are the indices of the positive pairs (the diagonal)
    labels = torch.arange(len(query), device=query.device)

    # Calculate the cross-entropy loss
    return F.cross_entropy(logits / temperature, labels)


class SemanticLoss(nn.Module):
    """
    A loss function that measures the semantic similarity between two sets of captions
    using a pre-trained SentenceTransformer model.
    """
    def __init__(self, device):
        super().__init__()
        self.device = device
        # Load a pre-trained model optimized for semantic similarity
        self.model = SentenceTransformer('all-MiniLM-L6-v2').to(device)
        # Freeze the model's weights as we only use it for inference
        for param in self.model.parameters():
            param.requires_grad = False

    def forward(self, predicted_captions, ground_truth_captions):
        # Convert the text captions into semantic embedding vectors
        pred_embeddings = self.model.encode(predicted_captions, convert_to_tensor=True)
        gt_embeddings = self.model.encode(ground_truth_captions, convert_to_tensor=True)

        # Calculate the cosine similarity. The loss is 1.0 minus the similarity.
        # A higher similarity (closer to 1.0) results in a lower loss (closer to 0.0).
        cosine_sim = F.cosine_similarity(pred_embeddings, gt_embeddings, dim=-1)
        loss = 1.0 - cosine_sim.mean()
        return loss

#------------------------------------------------------------------------------------------

def create_and_save_reconstruction_grid(eval_samples, diffusion_pipe, epoch, val_loss, output_dir):
    """
    Generates a 2x2 image grid comparing original and reconstructed images.
    Top row: Original images with ground truth captions.
    Bottom row: Reconstructed images with predicted captions.
    """
    if len(eval_samples) < 2:
        print("Not enough samples for a 2x2 grid, skipping.")
        return

    print(f"\n🖼️  Generating reconstruction grid for epoch {epoch}...")

    # Setup canvas and font
    w, h = 512, 512  # Standard Stable Diffusion size
    title_h = 60    # Space for titles
    grid = Image.new('RGB', (w * 2, h * 2 + title_h * 2), 'black')
    draw = ImageDraw.Draw(grid)
    try:
        font = ImageFont.truetype("LiberationSans-Regular.ttf", 20)
    except IOError:
        print("Default font not found, using fallback.")
        font = ImageFont.load_default()

    # Process two samples
    for i in range(5):
        predicted_cap, gt_cap, original_image = eval_samples[i]

        # --- Generate the reconstructed image from the predicted caption ---
        generator = torch.Generator(device=diffusion_pipe.device).manual_seed(42 + i)
        reconstructed_image = diffusion_pipe(
            prompt=predicted_cap, generator=generator, num_inference_steps=20
        ).images[0]

        # --- Paste images and draw titles ---
        col_offset = i * w

        # Top Row (Original)
        draw.text((col_offset + 5, 5), f"GT: {gt_cap[:50]}...", font=font, fill="white")
        grid.paste(original_image.resize((w, h)), (col_offset, title_h))

        # Bottom Row (Reconstructed)
        draw.text((col_offset + 5, h + title_h + 5), f"Pred: {predicted_cap[:50]}...", font=font, fill="white")
        grid.paste(reconstructed_image.resize((w, h)), (col_offset, h + title_h * 2))

    # --- Save the final grid ---
    # Sanitize loss value for filename
    val_loss_str = f"{val_loss:.4f}".replace('.', '_')
    save_path = Path(output_dir) / 'reconstructions'
    save_path.mkdir(exist_ok=True)
    filename = save_path / f"reconstruction_epoch_{epoch}_loss_{val_loss_str}.png"
    grid.save(filename)
    print(f"  Saved reconstruction grid to {filename}")

#----------------------------------------------------------------------------------------------
def train_model(config):
    # --- Setup (Device, Output Dir, etc.) ---
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    output_dir = Path(config.OUTPUT_DIR) / f'run_{timestamp}'
    output_dir.mkdir(parents=True, exist_ok=True)

    # --- Models and Data ---
    processor = Blip2Processor.from_pretrained(config.BLIP_MODEL_NAME)
    model = EEG_BLIP2_Model(config, device) # Assumes the refactored, cleaner version

    try:
      print('Loading best model weights')
      model.load_state_dict(torch.load(config.BEST_MODEL_DIR, map_location=device))
      print('Successfully loaded best model weights!')
    except:
      print('Could not load the best model weights.')

    print("Loading Stable Diffusion pipeline for visualizations...")
    diffusion_pipe = StableDiffusionPipeline.from_pretrained(
        config.DIFFUSION_MODEL_ID,
        torch_dtype=torch.float16 if device.type == "cuda" else torch.float32,
        safety_checker=None, requires_safety_checker=False
    ).to(device)
    diffusion_pipe.set_progress_bar_config(disable=True)

    semantic_loss_fn = SemanticLoss(device)

    # --- Data Loading ---
    spec_mean = torch.load(Path(config.PROCESSED_DATA_ROOT) / 'spec_mean.pt')
    spec_std = torch.load(Path(config.PROCESSED_DATA_ROOT) / 'spec_std.pt')
    spec_std += 1e-6

    transform = transforms.Compose([
        transforms.ToDtype(torch.float32),
        transforms.Resize((224, 224), antialias=True),
        transforms.Normalize(mean=spec_mean.tolist(), std=spec_std.tolist()),
    ])

    train_dataset = EEGDatasetWithCaptions(config.PROCESSED_DATA_ROOT, config.METADATA_CSV, 'train', transform, augment=True)
    val_dataset = EEGDatasetWithCaptions(config.PROCESSED_DATA_ROOT, config.METADATA_CSV, 'val', transform, augment=False)

    # --- Training Components ---
    scaler = torch.cuda.amp.GradScaler()
    best_val_loss = float('inf')

    generation_args = {
        "max_new_tokens": 30, "do_sample": True, "min_new_tokens": 8,
        "repetition_penalty": 1.25, "top_p": 0.9, "temperature": 0.81
    }

    # --- Main Training Loop ---
    print("🎯 Starting Decoupled 2-Stage Training Loop")
    for epoch in range(15, config.NUM_EPOCHS):
        # --- Stage 1 & 2 Freezing Logic ---
        if epoch < config.STAGE1_EPOCHS:
            if epoch == 0:
                print(f"\n--- STAGE 1: Training Adapter ONLY (Epochs 0-{config.STAGE1_EPOCHS-1}) ---")
            for param in model.adapter.parameters():
                param.requires_grad = True
            for param in model.blip.parameters():
                param.requires_grad = False
        elif epoch == config.STAGE1_EPOCHS:
            print(f"\n--- STAGE 2: Fine-Tuning Q-Former ONLY (Epochs {config.STAGE1_EPOCHS}+) ---")
            for param in model.adapter.parameters():
                param.requires_grad = False
            for param in model.blip.qformer.parameters():
                param.requires_grad = True
            for param in model.blip.vision_model.parameters():
                param.requires_grad = True
            if hasattr(model.blip, "language_projection"):
                for param in model.blip.language_projection.parameters():
                    param.requires_grad = True

        # --- Create/Re-create Optimizer & Scheduler at stage transitions ---
        if epoch == 0 or epoch == config.STAGE1_EPOCHS:
            print(f"Creating optimizer for Stage {1 if epoch < config.STAGE1_EPOCHS else 2}...")
            if epoch < config.STAGE1_EPOCHS:
                # Stage 1: Only adapter is trained
                optimizer = optim.AdamW(model.adapter.parameters(), lr=config.ADAPTER_LR, weight_decay=config.WEIGHT_DECAY)
            else:
                # Stage 2: Only BLIP layers are trained
                trainable_params = filter(lambda p: p.requires_grad, model.parameters())
                optimizer = optim.AdamW(trainable_params, lr=config.BLIP_LR, weight_decay=config.WEIGHT_DECAY)

            num_training_steps = (len(train_dataset) // config.BATCH_SIZE) * config.NUM_EPOCHS
            scheduler = get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=config.WARMUP_STEPS, num_training_steps=num_training_steps
            )
            print("Optimizer and scheduler created.")

        # --- Training Batch Loop ---
        model.train()
        train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE,
            sampler=SubsetRandomSampler(np.random.choice(len(train_dataset), min(len(train_dataset), config.TRAIN_SAMPLES_PER_EPOCH), replace=False)),
            drop_last=True, collate_fn=collate_fn)
        train_bar = tqdm(train_loader, desc=f"🔄 Epoch {epoch+1}/{config.NUM_EPOCHS}")

        for batch_idx, (spectrograms, pil_images, gt_captions) in enumerate(train_bar):
            spectrograms = spectrograms.to(device)
            with torch.amp.autocast('cuda'):
                eeg_embeddings = model.get_eeg_embedding(spectrograms)
                with torch.no_grad():
                    generated_ids = model.generate(spectrograms, **generation_args)
                    eeg_captions = processor.batch_decode(generated_ids, skip_special_tokens=True)
                    image_embeddings = model.get_image_embedding(pil_images, processor)

                loss_align = info_nce_loss(eeg_embeddings, image_embeddings)
                loss_semantic = semantic_loss_fn(eeg_captions, gt_captions)

                # Use consistent loss weights defined in config
                loss = (config.ALIGN_WEIGHT * loss_align) + (config.SEMANTIC_WEIGHT * loss_semantic)
                loss = loss / config.GRAD_ACCUMULATION_STEPS

            scaler.scale(loss).backward()

            if (batch_idx + 1) % config.GRAD_ACCUMULATION_STEPS == 0:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(filter(lambda p: p.requires_grad, model.parameters()), config.GRADIENT_CLIP_NORM)
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
                scheduler.step()

            train_bar.set_postfix({"align_loss": f"{loss_align.item():.4f}", "sem_loss": f"{loss_semantic.item():.4f}"})

        # --- Validation Loop ---
        print(f"\n🔍 Running Validation for epoch {epoch+1}")
        model.eval()

        # Lists to store metrics for this epoch
        val_align_losses = []
        val_semantic_losses = []
        eval_samples = [] # To store samples for printing and image generation

        # Create the validation data loader
        val_indices = np.random.choice(len(val_dataset), min(len(val_dataset), config.VAL_SAMPLES_PER_EPOCH), replace=False)
        val_loader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE,
            sampler=SubsetRandomSampler(val_indices), collate_fn=collate_fn)

        # Disable gradients for the validation loop
        with torch.no_grad(), torch.amp.autocast('cuda'):
            for spectrograms, pil_images, gt_captions in tqdm(val_loader, desc="Validation Loop"):
                spectrograms = spectrograms.to(device)

                # --- Perform the forward pass ---
                eeg_embeddings = model.get_eeg_embedding(spectrograms)
                image_embeddings = model.get_image_embedding(pil_images, processor)

                # Generate predicted captions from the EEG
                generated_ids = model.generate(spectrograms, **generation_args)
                eeg_captions = processor.batch_decode(generated_ids, skip_special_tokens=True)

                # --- Calculate the two primary losses ---
                loss_align = info_nce_loss(eeg_embeddings, image_embeddings)
                loss_semantic = semantic_loss_fn(eeg_captions, gt_captions)

                val_align_losses.append(loss_align.item())
                val_semantic_losses.append(loss_semantic.item())

                # --- Collect a few samples for visualization ---
                if len(eval_samples) < config.VIS_GRID_SIZE:
                    num_to_add = config.VIS_GRID_SIZE - len(eval_samples)
                    for i in range(min(num_to_add, len(eeg_captions))):
                        eval_samples.append((eeg_captions[i], gt_captions[i], pil_images[i]))

        avg_val_align_loss = np.mean(val_align_losses)
        avg_val_semantic_loss = np.mean(val_semantic_losses)

        # Use consistent loss weights
        total_val_loss = (config.ALIGN_WEIGHT * avg_val_align_loss) + (config.SEMANTIC_WEIGHT * avg_val_semantic_loss)

        print(f"  Validation Results:")
        print(f"    - Avg Alignment Loss: {avg_val_align_loss:.4f}")
        print(f"    - Avg Semantic Loss:  {avg_val_semantic_loss:.4f}")
        print(f"    - Total Weighted Loss: {total_val_loss:.4f}")

        # Update the learning rate scheduler based on the total validation loss
        # scheduler.step(total_val_loss)

        # --- Save the best model based on the total validation loss ---
        if total_val_loss < best_val_loss:
            best_val_loss = total_val_loss
            save_path = output_dir / f'best_model_epoch_{epoch+1}_loss_{total_val_loss:.4f}.pth'
            torch.save(model.state_dict(), save_path)
            print(f"🏆 New best model saved to {save_path}")

        # --- Print caption comparisons ---
        if eval_samples:
            print("\n--- ✍️  Caption Generation Samples ---")
            for i, (predicted_cap, gt_cap, _) in enumerate(eval_samples):
                print(f"Sample {i+1}:")
                print(f"  - Ground Truth: {gt_cap.strip()}")
                print(f"  - Predicted:    {predicted_cap.strip()}")
            print("------------------------------------")

        if (epoch + 1) % config.EVAL_IMAGE_GENERATION_INTERVAL == 0 and eval_samples:
            create_and_save_reconstruction_grid(
                eval_samples,
                diffusion_pipe,
                epoch + 1,
                total_val_loss,
                output_dir
            )

    print("\n🎉 Training Complete!")
    return output_dir

if __name__ == '__main__':
    config = TRAIN_CONFIG()
    try:
        output_dir = train_model(config)
        if output_dir:
            print(f"✅ Training completed successfully! Results saved to: {output_dir}")
    except Exception as e:
        print(f"❌ Training failed with error: {str(e)}")
        import traceback
        traceback.print_exc()

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading best model weights
Successfully loaded best model weights!
Loading Stable Diffusion pipeline for visualizations...


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

...


🔍 Running Validation for epoch 34


Validation Loop:   0%|          | 0/50 [00:00<?, ?it/s]

  Validation Results:
    - Avg Alignment Loss: 2.0910
    - Avg Semantic Loss:  0.8927
    - Total Weighted Loss: 4.3227

--- ✍️  Caption Generation Samples ---
Sample 1:
  - Ground Truth: a bed with white sheets
  - Predicted:    a black and grey background with the word "breathtaking"
Sample 2:
  - Ground Truth: three zebras grazing in a field near a body of water
  - Predicted:    a large pink, blue and purple line art
Sample 3:
  - Ground Truth: a man sitting at a table eating a meal
  - Predicted:    a red and gray pattern of a large floor
Sample 4:
  - Ground Truth: a large air france airplane flying through the air
  - Predicted:    a pink and brown line on a grey background
Sample 5:
  - Ground Truth: a bathroom with a toilet, sink and mirror
  - Predicted:    a collection of a pink and purple
------------------------------------

🖼️  Generating reconstruction grid for epoch 34...
  Saved reconstruction grid to /content/drive/MyDrive/NeuroVision/EEG_BLIP2_Alignment/run_2025100

🔄 Epoch 35/50:   0%|          | 0/250 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
from pathlib import Path
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torchvision.transforms import v2 as transforms
from PIL import Image
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.auto import tqdm
from datetime import datetime
import numpy as np
from transformers import Blip2Processor, Blip2ForConditionalGeneration, Blip2VisionModel
import warnings
from sentence_transformers import SentenceTransformer
from diffusers import StableDiffusionPipeline
import torch.nn.functional as F
from PIL import Image, ImageDraw, ImageFont
import json
from transformers import get_linear_schedule_with_warmup

# ==============================================================================
# --- 1. CONFIGURATION ---
# ==============================================================================
class TRAIN_CONFIG:
    PROCESSED_DATA_ROOT = '/content/final_lightweight_17k'
    METADATA_CSV = Path(PROCESSED_DATA_ROOT) / 'metadata.csv'
    OUTPUT_DIR = '/content/drive/MyDrive/NeuroVision/EEG_BLIP2_Alignment'

    BLIP_MODEL_NAME = "Salesforce/blip2-opt-2.7b"

    BATCH_SIZE = 16
    GRAD_ACCUMULATION_STEPS = 2
    NUM_EPOCHS = 40
    ADAPTER_LR = 1e-5  # Higher LR for the new adapter
    BLIP_LR = 3e-6     # Lower LR for fine-tuning the pre-trained layers

    WEIGHT_DECAY = 1e-3
    GRADIENT_CLIP_NORM = 1.0

    VALIDATION_INTERVAL = 1
    VAL_SAMPLES_PER_EPOCH = 700
    TRAIN_SAMPLES_PER_EPOCH = 3500
    VIS_GRID_SIZE = 5

    DIFFUSION_MODEL_ID ="runwayml/stable-diffusion-v1-5"
    EVAL_IMAGE_GENERATION_INTERVAL = 1
    WARMUP_STEPS = 150  # Number of steps to gradually increase the LR
    TOTAL_TRAIN_STEPS = 218 * 40
    STAGE1_EPOCHS = 2

# ==============================================================================
# --- 2. EEG ENCODER MODULE ---
# ==============================================================================

class ChannelAdapter(nn.Module):
    """
    A powerful CNN adapter inspired by EEGNet principles to extract robust
    features from EEG spectrograms. It processes the 64 channels to find
    temporal and spatial patterns, then creates a 3-channel pseudo-image.
    """
    def __init__(self, in_chans=64, out_chans=3, F1=16, D=2, F2=32, dropout=0.4):
        super().__init__()

        # Block 1: Temporal and Depthwise Spatial Convolutions
        self.block1 = nn.Sequential(
            # Input is [B, 1, Chans, Samples]
            # Temporal Conv to learn filters across time
            nn.Conv2d(1, F1, (1, 64), padding='same', bias=False),
            nn.BatchNorm2d(F1),
            # Depthwise Spatial Conv to learn spatial filters for each feature map
            nn.Conv2d(F1, F1 * D, (in_chans, 1), groups=F1, bias=False),
            nn.BatchNorm2d(F1 * D),
            nn.ELU(),
            # nn.AvgPool2d((1, 4)),
            nn.Dropout(dropout)
        )

        # Block 2: Separable Convolutions for more abstract features
        self.block2 = nn.Sequential(
            # Separable Conv (Depthwise followed by Pointwise)
            nn.Conv2d(F1 * D, F2, (1, 16), padding='same', groups=F1 * D, bias=False),
            nn.Conv2d(F2, F2, (1, 1), bias=False),
            nn.BatchNorm2d(F2),
            nn.ELU(),
            # nn.AvgPool2d((1, 8)),
            nn.Dropout(dropout)
        )

        # Final projection layer to create the 3-channel "pseudo-image"
        self.final_projection = nn.Conv2d(F2, out_chans, (1, 1))
        self.adaptive_pool = nn.AdaptiveAvgPool2d((224, 224))

    def forward(self, x):
        # Input x is [Batch, 64 Channels, Freq Bins, Time Steps]
        # We reshape it to fit the EEGNet structure: [Batch, 1, Channels, Samples]
        # This treats the frequency and time dimensions as one long sequence.
        x = x.view(x.size(0), 1, x.size(1), -1)

        x = self.block1(x)
        x = self.block2(x)

        # Project to 3 channels
        x = self.final_projection(x)
        x = self.adaptive_pool(x)

        # The output is a feature map, which will be resized by the vision model's patch embedding
        return x
#-------------------------------------------------------------------------------------------

class EEGEncoder(nn.Module):
    def __init__(self, config, in_chans=64):
        super().__init__()
        # self.vision_model = Blip2VisionModel.from_pretrained(config.BLIP_MODEL_NAME)

        # Freeze the original vision model
        # for param in self.vision_model.parameters():
        #     param.requires_grad = False

        self.adapter = ChannelAdapter(in_chans=in_chans)

        for p in self.adapter.parameters():
            p.requires_grad = True


    def forward(self, x, vision_model):
      adapted_x = self.adapter(x)
      vision_outputs = vision_model(adapted_x, return_dict=True)
      return vision_outputs.last_hidden_state

# ==============================================================================
# --- 3. THE HYBRID EEG-BLIP2 MODEL ---
# ==============================================================================
class EEG_BLIP2_Model(nn.Module):
    def __init__(self, config, device):
        super().__init__()
        self.device = device
        self.eeg_encoder = EEGEncoder(config)
        self.blip = Blip2ForConditionalGeneration.from_pretrained(
            config.BLIP_MODEL_NAME
        )

        # Freeze the entire BLIP model initially
        for param in self.blip.parameters():
            param.requires_grad = False

        # print("Unfreezing specified layers for fine-tuning...")

        # --- THIS IS THE KEY CHANGE ---
        # Unfreeze the ENTIRE Q-Former
        # for param in self.blip.qformer.parameters():
        #     param.requires_grad = True
        # print(" - Unfroze the entire Q-Former.")

        # # Also unfreeze the language projection layer (connects Q-Former to LLM)
        # if hasattr(self.blip, "language_projection"):
        #     for param in self.blip.language_projection.parameters():
        #         param.requires_grad = True
        #     print("  - Unfroze Language Projection layer.")


        self.eeg_encoder.to(device)
        self.blip.to(device)

    def get_eeg_embedding(self, eeg_spectrograms):
        eeg_features = self.eeg_encoder(eeg_spectrograms, self.blip.vision_model)
        query_tokens = self.blip.query_tokens.expand(eeg_features.shape[0], -1, -1)
        query_outputs = self.blip.qformer(
            query_embeds=query_tokens,
            encoder_hidden_states=eeg_features,
            return_dict=True,
        )
        return query_outputs.last_hidden_state.mean(dim=1)

    def get_image_embedding(self, pil_images, processor):
        inputs = processor(images=pil_images, return_tensors="pt").to(self.device)
        pixel_values = inputs.pixel_values

        image_features = self.blip.vision_model(pixel_values).last_hidden_state
        query_tokens = self.blip.query_tokens.expand(image_features.shape[0], -1, -1)
        query_outputs = self.blip.qformer(
            query_embeds=query_tokens,
            encoder_hidden_states=image_features,
            return_dict=True,
        )
        return query_outputs.last_hidden_state.mean(dim=1)

    def generate(self, eeg_spectrograms, **kwargs):
        with torch.no_grad():
            eeg_features = self.eeg_encoder(eeg_spectrograms, self.blip.vision_model)
            query_tokens = self.blip.query_tokens.expand(eeg_features.shape[0], -1, -1)
            query_outputs = self.blip.qformer(
                query_embeds=query_tokens,
                encoder_hidden_states=eeg_features,
                return_dict=True,
            )
            language_model_inputs = self.blip.language_projection(query_outputs.last_hidden_state)
            generated_ids = self.blip.language_model.generate(
                inputs_embeds=language_model_inputs,
                **kwargs
            )
            return generated_ids

# ==============================================================================
# --- 4. DATASET & COLLATING ---
# ==============================================================================
def collate_fn(batch):
    spectrograms = torch.stack([item[0] for item in batch])
    pil_images = [item[1] for item in batch]
    captions = [item[2] for item in batch]
    return spectrograms, pil_images, captions

class EEGDatasetWithCaptions(Dataset):
    def __init__(self, root_dir, metadata_csv, split, transform, augment=False):
        self.root_dir = Path(root_dir)
        self.transform = transform
        df = pd.read_csv(metadata_csv)
        self.split_df = df[df['split'].str.strip() == split].reset_index(drop=True)
        self.augment = augment

        # --- NEW: Define a dedicated augmentation pipeline ---
        if self.augment:
            self.augmentation_transform = transforms.Compose([
                transforms.RandomApply([transforms.GaussianBlur(kernel_size=3)], p=0.5),
                # Randomly mask out frequency bands
                transforms.RandomApply([transforms.RandomErasing(p=1.0, scale=(0.02, 0.1), ratio=(0.1, 0.5))], p=0.5),
                # Randomly mask out time steps
                transforms.RandomApply([transforms.RandomErasing(p=1.0, scale=(0.02, 0.1), ratio=(2.0, 5.0))], p=0.5),
            ])

        # This is your existing normalization transform
        self.normalization_transform = transform

        # Load pre-generated captions
        captions_path = Path(root_dir) / f'{split}_gt_captions.json'
        print(f"Loading pre-generated captions from {captions_path}")
        with open(captions_path, 'r') as f:
            self.captions = json.load(f)

    def __len__(self):
        return len(self.split_df)

    def __getitem__(self, idx):
        info = self.split_df.iloc[idx]

        spectrogram = torch.load(self.root_dir / info['spectrogram_path'])

        # --- APPLY AUGMENTATIONS (for training set only) ---
        if self.augment:
            spectrogram = self.augmentation_transform(spectrogram)

        # Apply normalization after augmentation
        spectrogram = self.normalization_transform(spectrogram)

        image = Image.open(self.root_dir / info['image_path']).convert("RGB")
        gt_caption = self.captions.get(str(idx), "an image")

        return spectrogram, image, gt_caption

# ==============================================================================
# --- 5. MAIN TRAINING FUNCTION ---
# ==============================================================================
# InfoNCE loss function

def info_nce_loss(query, positive_key, temperature=0.07):
    # Ensure inputs are normalized
    query = F.normalize(query, dim=-1)
    positive_key = F.normalize(positive_key, dim=-1)

    # Calculate the similarity matrix of every query with every key
    # The diagonal of this matrix contains the positive pairs
    logits = query @ positive_key.T

    # The labels are the indices of the positive pairs (the diagonal)
    labels = torch.arange(len(query), device=query.device)

    # Calculate the cross-entropy loss
    return F.cross_entropy(logits / temperature, labels)


class SemanticLoss(nn.Module):
    """
    A loss function that measures the semantic similarity between two sets of captions
    using a pre-trained SentenceTransformer model.
    """
    def __init__(self, device):
        super().__init__()
        self.device = device
        # Load a pre-trained model optimized for semantic similarity
        self.model = SentenceTransformer('all-MiniLM-L6-v2').to(device)
        # Freeze the model's weights as we only use it for inference
        for param in self.model.parameters():
            param.requires_grad = False

    def forward(self, predicted_captions, ground_truth_captions):
        # Convert the text captions into semantic embedding vectors
        pred_embeddings = self.model.encode(predicted_captions, convert_to_tensor=True)
        gt_embeddings = self.model.encode(ground_truth_captions, convert_to_tensor=True)

        # Calculate the cosine similarity. The loss is 1.0 minus the similarity.
        # A higher similarity (closer to 1.0) results in a lower loss (closer to 0.0).
        cosine_sim = F.cosine_similarity(pred_embeddings, gt_embeddings, dim=-1)
        loss = 1.0 - cosine_sim.mean()
        return loss

#------------------------------------------------------------------------------------------

def create_and_save_reconstruction_grid(eval_samples, diffusion_pipe, epoch, val_loss, output_dir):
    """
    Generates a 2x2 image grid comparing original and reconstructed images.
    Top row: Original images with ground truth captions.
    Bottom row: Reconstructed images with predicted captions.
    """
    if len(eval_samples) < 2:
        print("Not enough samples for a 2x2 grid, skipping.")
        return

    print(f"\n🖼️  Generating reconstruction grid for epoch {epoch}...")

    # Setup canvas and font
    w, h = 512, 512  # Standard Stable Diffusion size
    title_h = 60    # Space for titles
    grid = Image.new('RGB', (w * 2, h * 2 + title_h * 2), 'black')
    draw = ImageDraw.Draw(grid)
    try:
        font = ImageFont.truetype("LiberationSans-Regular.ttf", 20)
    except IOError:
        print("Default font not found, using fallback.")
        font = ImageFont.load_default()

    # Process two samples
    for i in range(5):
        predicted_cap, gt_cap, original_image = eval_samples[i]

        # --- Generate the reconstructed image from the predicted caption ---
        generator = torch.Generator(device=diffusion_pipe.device).manual_seed(42 + i)
        reconstructed_image = diffusion_pipe(
            prompt=predicted_cap, generator=generator, num_inference_steps=20
        ).images[0]

        # --- Paste images and draw titles ---
        col_offset = i * w

        # Top Row (Original)
        draw.text((col_offset + 5, 5), f"GT: {gt_cap[:50]}...", font=font, fill="white")
        grid.paste(original_image.resize((w, h)), (col_offset, title_h))

        # Bottom Row (Reconstructed)
        draw.text((col_offset + 5, h + title_h + 5), f"Pred: {predicted_cap[:50]}...", font=font, fill="white")
        grid.paste(reconstructed_image.resize((w, h)), (col_offset, h + title_h * 2))

    # --- Save the final grid ---
    # Sanitize loss value for filename
    val_loss_str = f"{val_loss:.4f}".replace('.', '_')
    save_path = Path(output_dir) / 'reconstructions'
    save_path.mkdir(exist_ok=True)
    filename = save_path / f"reconstruction_epoch_{epoch}_loss_{val_loss_str}.png"
    grid.save(filename)
    print(f"  Saved reconstruction grid to {filename}")

#----------------------------------------------------------------------------------------------
def train_model(config):
    # --- Setup (Device, Output Dir, etc.) ---
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    output_dir = Path(config.OUTPUT_DIR) / f'run_{timestamp}'
    output_dir.mkdir(parents=True, exist_ok=True)

    # --- Models and Data ---
    processor = Blip2Processor.from_pretrained(config.BLIP_MODEL_NAME)
    model = EEG_BLIP2_Model(config, device)
    try:
        model.load_state_dict(torch.load('/content/drive/MyDrive/NeuroVision/EEG_BLIP2_Alignment/run_20250930_190044/best_model_epoch_10_loss_5.9745.pth',
                                         map_location=device))
        print('Loaded saved EEG_BLIP2_Model weights')
    except:
        print('Could not load eeg_blip2_model weights')

    print("Loading Stable Diffusion pipeline for visualizations...")
    diffusion_pipe = StableDiffusionPipeline.from_pretrained(
        config.DIFFUSION_MODEL_ID,
        torch_dtype=torch.float16 if device.type == "cuda" else torch.float32,
        safety_checker=None, requires_safety_checker=False
    ).to(device)
    diffusion_pipe.set_progress_bar_config(disable=True)

    semantic_loss_fn = SemanticLoss(device)

    # --- Data Loading (with Epsilon fix) ---
    spec_mean = torch.load(Path(config.PROCESSED_DATA_ROOT) / 'spec_mean.pt')
    spec_std = torch.load(Path(config.PROCESSED_DATA_ROOT) / 'spec_std.pt')
    spec_std += 1e-6

    transform = transforms.Compose([
        transforms.ToDtype(torch.float32),
        transforms.Resize((224, 224), antialias=True),
        transforms.Normalize(mean=spec_mean.tolist(), std=spec_std.tolist()),
    ])

    train_dataset = EEGDatasetWithCaptions(config.PROCESSED_DATA_ROOT, config.METADATA_CSV, 'train', transform, augment=True)
    val_dataset = EEGDatasetWithCaptions(config.PROCESSED_DATA_ROOT, config.METADATA_CSV, 'val', transform, augment=False)

    # --- Optimizer, Scheduler, and Training Components ---
    # NOTE: Moved outside the loop for correct initialization
    scaler = torch.cuda.amp.GradScaler()
    best_val_loss = float('inf')

    generation_args = {
        "max_new_tokens": 30, "do_sample": True, "min_new_tokens": 6,
        "repetition_penalty": 1.4, "top_p": 0.9, "temperature": 0.85
    }

    # --- Main Training Loop ---
    print("🎯 Starting 2-Stage Training Loop")
    for epoch in range(config.NUM_EPOCHS):
        # --- STAGE 1: Train Adapter Only (Epochs 0-19) ---
        if epoch < config.STAGE1_EPOCHS:
            if epoch == 0:
                print("\n--- STAGE 1: Training EEGNetAdapter Only ---")
            for param in model.eeg_encoder.adapter.parameters():
                param.requires_grad = True
            for param in model.blip.parameters():
                param.requires_grad = False

        # --- STAGE 2: Fine-Tune Q-Former (Epochs 20+) ---
        elif epoch == config.STAGE1_EPOCHS:
            print("\n--- STAGE 2: Fine-Tuning Q-Former ---")
            for param in model.blip.qformer.parameters():
                param.requires_grad = True
            if hasattr(model.blip, "language_projection"):
                for param in model.blip.language_projection.parameters():
                    param.requires_grad = True
                print("  - Unfroze Language Projection layer.")

        # --- Create Optimizer & Scheduler at the start of each stage ---
        if epoch == 0 or epoch == config.STAGE1_EPOCHS:
            print(f"Creating optimizer for Stage {1 if epoch==0 else 2}...")
            adapter_params = [p for n, p in model.named_parameters() if p.requires_grad and 'adapter' in n]
            blip_params = [p for n, p in model.named_parameters() if p.requires_grad and 'adapter' not in n]

            optimizer_grouped_parameters = [
                {"params": adapter_params, "lr": config.ADAPTER_LR},
                {"params": blip_params, "lr": config.BLIP_LR},
            ]
            optimizer = optim.AdamW(optimizer_grouped_parameters, weight_decay=config.WEIGHT_DECAY)

            # Calculate total steps for the whole training run for the scheduler
            num_training_steps = (len(train_dataset) // config.BATCH_SIZE) * config.NUM_EPOCHS
            scheduler = get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=300, num_training_steps=num_training_steps
            )
            print("Optimizer and scheduler created.")

        model.train()
        train_loader = DataLoader(
            train_dataset, batch_size=config.BATCH_SIZE,
            sampler=SubsetRandomSampler(np.random.choice(len(train_dataset), min(len(train_dataset), config.TRAIN_SAMPLES_PER_EPOCH), replace=False)),
            drop_last=True, collate_fn=collate_fn
        )
        train_bar = tqdm(train_loader, desc=f"🔄 Epoch {epoch+1}/{config.NUM_EPOCHS}")

        for batch_idx, (spectrograms, pil_images, gt_captions) in enumerate(train_bar):
            spectrograms = spectrograms.to(device)

            with torch.amp.autocast('cuda'):
                eeg_embeddings = model.get_eeg_embedding(spectrograms)
                with torch.no_grad():
                    generated_ids = model.generate(spectrograms, **generation_args)
                    eeg_captions = processor.batch_decode(generated_ids, skip_special_tokens=True)
                    image_embeddings = model.get_image_embedding(pil_images, processor)

                loss_align = info_nce_loss(eeg_embeddings, image_embeddings)
                loss_semantic = semantic_loss_fn(eeg_captions, gt_captions)
                loss = (1.0 * loss_align) + (2.5 * loss_semantic)
                loss = loss / config.GRAD_ACCUMULATION_STEPS

            # scaler.scale() must be outside the autocast block
            scaler.scale(loss).backward()

            # Optimizer step logic must also be outside the autocast block
            if (batch_idx + 1) % config.GRAD_ACCUMULATION_STEPS == 0:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(filter(lambda p: p.requires_grad, model.parameters()), config.GRADIENT_CLIP_NORM)
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
                scheduler.step()

            train_bar.set_postfix({"align_loss": f"{loss_align.item():.4f}", "sem_loss": f"{loss_semantic.item():.4f}"})


        print(f"\n🔍 Running Validation for epoch {epoch+1}")
        model.eval()

        # Lists to store metrics for this epoch
        val_align_losses = []
        val_semantic_losses = []
        eval_samples = [] # To store samples for printing and image generation

        # Create the validation data loader
        val_indices = np.random.choice(len(val_dataset), min(len(val_dataset), config.VAL_SAMPLES_PER_EPOCH), replace=False)
        val_loader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE, sampler=SubsetRandomSampler(val_indices), collate_fn=collate_fn)

        # Disable gradients for the validation loop
        with torch.no_grad(), torch.amp.autocast('cuda'):
            for spectrograms, pil_images, gt_captions in tqdm(val_loader, desc="Validation Loop"):
                spectrograms = spectrograms.to(device)

                # --- Perform the forward pass ---
                eeg_embeddings = model.get_eeg_embedding(spectrograms)
                image_embeddings = model.get_image_embedding(pil_images, processor)

                # Generate predicted captions from the EEG
                generated_ids = model.generate(spectrograms, **generation_args)
                eeg_captions = processor.batch_decode(generated_ids, skip_special_tokens=True)

                # --- Calculate the two primary losses ---
                loss_align = info_nce_loss(eeg_embeddings, image_embeddings)
                loss_semantic = semantic_loss_fn(eeg_captions, gt_captions)

                val_align_losses.append(loss_align.item())
                val_semantic_losses.append(loss_semantic.item())

                # --- Collect a few samples for visualization ---
                if len(eval_samples) < config.VIS_GRID_SIZE:
                    num_to_add = config.VIS_GRID_SIZE - len(eval_samples)
                    for i in range(min(num_to_add, len(eeg_captions))):
                        eval_samples.append((eeg_captions[i], gt_captions[i], pil_images[i]))

        # --- Calculate and print average losses ---
        avg_val_align_loss = np.mean(val_align_losses)
        avg_val_semantic_loss = np.mean(val_semantic_losses)
        # The total loss is a weighted sum, same as in the training loop
        total_val_loss = (1.0 * avg_val_align_loss) + (2.5 * avg_val_semantic_loss)

        print(f"  Validation Results:")
        print(f"    - Avg Alignment Loss: {avg_val_align_loss:.4f}")
        print(f"    - Avg Semantic Loss:  {avg_val_semantic_loss:.4f}")
        print(f"    - Total Weighted Loss: {total_val_loss:.4f}")

        # Update the learning rate scheduler based on the total validation loss
        # scheduler.step(total_val_loss)

        # --- Save the best model based on the total validation loss ---
        if total_val_loss < best_val_loss:
            best_val_loss = total_val_loss
            save_path = output_dir / f'best_model_epoch_{epoch+1}_loss_{total_val_loss:.4f}.pth'
            torch.save(model.state_dict(), save_path)
            print(f"🏆 New best model saved to {save_path}")

        # --- Print caption comparisons ---
        if eval_samples:
            print("\n--- ✍️  Caption Generation Samples ---")
            for i, (predicted_cap, gt_cap, _) in enumerate(eval_samples):
                print(f"Sample {i+1}:")
                print(f"  - Ground Truth: {gt_cap.strip()}")
                print(f"  - Predicted:    {predicted_cap.strip()}")
            print("------------------------------------")

        if (epoch + 1) % config.EVAL_IMAGE_GENERATION_INTERVAL == 0 and eval_samples:
            create_and_save_reconstruction_grid(
                eval_samples,
                diffusion_pipe,
                epoch + 1,
                total_val_loss,
                output_dir
            )

    print("\n🎉 Training Complete!")
    return output_dir

if __name__ == '__main__':
    config = TRAIN_CONFIG()
    try:
        output_dir = train_model(config)
        if output_dir:
            print(f"✅ Training completed successfully! Results saved to: {output_dir}")
    except Exception as e:
        print(f"❌ Training failed with error: {str(e)}")
        import traceback
        traceback.print_exc()

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded saved EEG_BLIP2_Model weights
Loading Stable Diffusion pipeline for visualizations...


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

...

🔄 Epoch 6/40:   0%|          | 0/218 [00:00<?, ?it/s]


🔍 Running Validation for epoch 6


Validation Loop:   0%|          | 0/44 [00:00<?, ?it/s]

  Validation Results:
    - Avg Alignment Loss: 2.7855
    - Avg Semantic Loss:  0.8981
    - Total Weighted Loss: 5.0306

--- ✍️  Caption Generation Samples ---
Sample 1:
  - Ground Truth: a pair of green scissors
  - Predicted:    the background of a black and white screen, with the image showing an animated red diagonal line
Sample 2:
  - Ground Truth: a green and white fire hydrant
  - Predicted:    a screenshot of a computer screen with an animated background
Sample 3:
  - Ground Truth: a bed with a white sheet
  - Predicted:    the video is shot with a red and white background
Sample 4:
  - Ground Truth: a double decker bus driving down a street
  - Predicted:    a pink and grey line on the background
Sample 5:
  - Ground Truth: a bear is walking through a field of green bushes
  - Predicted:    a purple and orange striped background
------------------------------------

🖼️  Generating reconstruction grid for epoch 6...
  Saved reconstruction grid to /content/drive/MyDrive/NeuroV

🔄 Epoch 7/40:   0%|          | 0/218 [00:00<?, ?it/s]

# Evaluations

In [ ]:
import torch
import torch.nn.functional as F
from pathlib import Path
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from torchvision.transforms import v2 as transforms
from torch.utils.data import DataLoader, SubsetRandomSampler
import numpy as np
from tqdm.auto import tqdm
import json
from transformers import Blip2Processor
from diffusers import StableDiffusionPipeline
import clip
from sentence_transformers import SentenceTransformer

class ModelEvaluator:
    """Comprehensive evaluation of trained EEG-to-Caption model"""

    def __init__(self, model_path, config, device):
        self.device = device
        self.config = config
        self.model_path = model_path

        # Load processor
        self.processor = Blip2Processor.from_pretrained(config.BLIP_MODEL_NAME)

        # Load trained model
        self.model = self._load_model()

        # Initialize evaluation tools
        self.sentence_model = SentenceTransformer('all-MiniLM-L6-v2').to(device)
        self.clip_model, self.clip_preprocess = clip.load(config.CLIP_MODEL_NAME, device=device)

        # Initialize Stable Diffusion
        print("Loading Stable Diffusion pipeline...")
        self.diffusion_pipe = StableDiffusionPipeline.from_pretrained(
            config.DIFFUSION_MODEL_ID,
            torch_dtype=torch.float16 if device == "cuda" else torch.float32,
            safety_checker=None,
            requires_safety_checker=False
        )
        self.diffusion_pipe = self.diffusion_pipe.to(device)
        self.diffusion_pipe.set_progress_bar_config(disable=True)

        # Initialize ground truth caption generator
        self.caption_blip = Blip2ForConditionalGeneration.from_pretrained(config.BLIP_MODEL_NAME)
        self.caption_blip.to(device)
        self.caption_blip.eval()

        print("✅ Model evaluator initialized successfully!")

    def _load_model(self):
        """Load the trained model"""
        # Create model
        model = EnhancedEEG_BLIP2_Model(self.config, self.device)

        # Load state dict
        checkpoint = torch.load(self.model_path, map_location=self.device)
        if 'model_state_dict' in checkpoint:
            model.load_state_dict(checkpoint['model_state_dict'])
            print(f"✅ Loaded model from epoch {checkpoint.get('epoch', 'unknown')}")
        else:
            model.load_state_dict(checkpoint)

        model.eval()
        return model

    def generate_captions_from_eeg(self, spectrograms, generation_params=None):
        """Generate captions from EEG spectrograms"""
        if generation_params is None:
            generation_params = self.config.GENERATION_PARAMS

        with torch.no_grad():
            generated_ids = self.model.generate(spectrograms, **generation_params)
            captions = self.processor.batch_decode(generated_ids, skip_special_tokens=True)
            return [cap.strip() for cap in captions]

    def generate_ground_truth_captions(self, images):
        """Generate ground truth captions from images"""
        with torch.no_grad():
            inputs = self.processor(images=images, return_tensors="pt").to(self.device)
            generated_ids = self.caption_blip.generate(
                **inputs,
                max_length=50,
                num_beams=3,
                temperature=0.8,
                do_sample=True
            )
            captions = self.processor.batch_decode(generated_ids, skip_special_tokens=True)
            return [cap.strip() for cap in captions]

    def compute_semantic_similarity(self, captions1, captions2):
        """Compute semantic similarity between two sets of captions"""
        embeddings1 = self.sentence_model.encode(captions1, convert_to_tensor=True)
        embeddings2 = self.sentence_model.encode(captions2, convert_to_tensor=True)

        similarities = F.cosine_similarity(embeddings1, embeddings2, dim=-1)
        return similarities.cpu().numpy()

    def compute_clip_score(self, images, captions):
        """Compute CLIP score between images and captions"""
        # Preprocess images
        if isinstance(images[0], Image.Image):
            image_inputs = torch.stack([self.clip_preprocess(img) for img in images]).to(self.device)
        else:
            image_inputs = images

        # Tokenize texts
        text_inputs = clip.tokenize(captions, truncate=True).to(self.device)

        with torch.no_grad():
            image_features = self.clip_model.encode_image(image_inputs)
            text_features = self.clip_model.encode_text(text_inputs)

            # Normalize features
            image_features = F.normalize(image_features, dim=-1)
            text_features = F.normalize(text_features, dim=-1)

            # Compute similarities
            similarities = torch.sum(image_features * text_features, dim=-1)
            return similarities.cpu().numpy()

    def generate_image_from_caption(self, caption, seed=42):
        """Generate image from caption using Stable Diffusion"""
        try:
            generator = torch.Generator(device=self.device).manual_seed(seed)
            with torch.no_grad():
                result = self.diffusion_pipe(
                    prompt=caption,
                    generator=generator,
                    num_inference_steps=25,
                    guidance_scale=7.5,
                    height=512,
                    width=512
                )
                return result.images[0]
        except Exception as e:
            print(f"Error generating image: {e}")
            return Image.new('RGB', (512, 512), color='black')

    def compute_image_reconstruction_score(self, original_images, eeg_captions):
        """Compute reconstruction quality by generating images from EEG captions"""
        reconstruction_scores = []

        print("🖼️  Evaluating image reconstruction quality...")
        for i, (original_image, caption) in enumerate(zip(original_images, eeg_captions)):
            # Generate image from EEG caption
            generated_image = self.generate_image_from_caption(caption, seed=42+i)

            # Compute CLIP similarity between original and generated images
            orig_tensor = self.clip_preprocess(original_image).unsqueeze(0).to(self.device)
            gen_tensor = self.clip_preprocess(generated_image).unsqueeze(0).to(self.device)

            with torch.no_grad():
                orig_features = self.clip_model.encode_image(orig_tensor)
                gen_features = self.clip_model.encode_image(gen_tensor)

                orig_features = F.normalize(orig_features, dim=-1)
                gen_features = F.normalize(gen_features, dim=-1)

                similarity = F.cosine_similarity(orig_features, gen_features, dim=-1)
                reconstruction_scores.append(similarity.item())

        return reconstruction_scores

    def comprehensive_evaluation(self, test_loader, num_samples=100, save_dir=None):
        """Run comprehensive evaluation on test data"""
        self.model.eval()

        if save_dir:
            save_dir = Path(save_dir)
            save_dir.mkdir(parents=True, exist_ok=True)

        all_results = {
            'semantic_similarities': [],
            'clip_scores_eeg': [],
            'clip_scores_gt': [],
            'reconstruction_scores': [],
            'sample_comparisons': []
        }

        sample_count = 0

        print(f"🔬 Running comprehensive evaluation on {num_samples} samples...")

        with torch.no_grad():
            for batch_idx, (spectrograms, pil_images, gt_captions) in enumerate(tqdm(test_loader)):
                if sample_count >= num_samples:
                    break

                spectrograms = spectrograms.to(self.device)
                batch_size = len(spectrograms)

                # Generate EEG captions
                eeg_captions = self.generate_captions_from_eeg(spectrograms)

                # Generate ground truth captions (if not provided)
                if not gt_captions or all(cap == "" for cap in gt_captions):
                    gt_captions = self.generate_ground_truth_captions(pil_images)

                # Compute semantic similarities
                semantic_sims = self.compute_semantic_similarity(eeg_captions, gt_captions)
                all_results['semantic_similarities'].extend(semantic_sims)

                # Compute CLIP scores
                clip_scores_eeg = self.compute_clip_score(pil_images, eeg_captions)
                clip_scores_gt = self.compute_clip_score(pil_images, gt_captions)
                all_results['clip_scores_eeg'].extend(clip_scores_eeg)
                all_results['clip_scores_gt'].extend(clip_scores_gt)

                # Compute reconstruction scores (only for first few samples due to time)
                if sample_count < 20:
                    reconstruction_scores = self.compute_image_reconstruction_score(
                        pil_images[:min(4, batch_size)],
                        eeg_captions[:min(4, batch_size)]
                    )
                    all_results['reconstruction_scores'].extend(reconstruction_scores)

                    # Save visual examples
                    if save_dir and sample_count < 8:
                        self._save_evaluation_examples(
                            pil_images[:min(2, batch_size)],
                            eeg_captions[:min(2, batch_size)],
                            gt_captions[:min(2, batch_size)],
                            semantic_sims[:min(2, batch_size)],
                            save_dir,
                            sample_count
                        )

                # Store sample comparisons
                for i in range(min(batch_size, num_samples - sample_count)):
                    if len(all_results['sample_comparisons']) < 50:  # Store up to 50 examples
                        all_results['sample_comparisons'].append({
                            'eeg_caption': eeg_captions[i],
                            'gt_caption': gt_captions[i],
                            'semantic_similarity': semantic_sims[i],
                            'clip_score_eeg': clip_scores_eeg[i],
                            'clip_score_gt': clip_scores_gt[i]
                        })

                sample_count += batch_size

        # Compute final statistics
        results_summary = self._compute_evaluation_statistics(all_results)

        # Save detailed results
        if save_dir:
            with open(save_dir / 'evaluation_results.json', 'w') as f:
                json.dump({
                    'summary': results_summary,
                    'detailed_results': all_results
                }, f, indent=2, default=lambda x: x.tolist() if isinstance(x, np.ndarray) else x)

        return results_summary, all_results

    def _compute_evaluation_statistics(self, results):
        """Compute evaluation statistics"""
        semantic_sims = np.array(results['semantic_similarities'])
        clip_eeg = np.array(results['clip_scores_eeg'])
        clip_gt = np.array(results['clip_scores_gt'])
        reconstruction = np.array(results['reconstruction_scores']) if results['reconstruction_scores'] else np.array([])

        stats = {
            'semantic_similarity': {
                'mean': float(np.mean(semantic_sims)),
                'std': float(np.std(semantic_sims)),
                'median': float(np.median(semantic_sims)),
                'min': float(np.min(semantic_sims)),
                'max': float(np.max(semantic_sims)),
                'high_quality_ratio': float(np.mean(semantic_sims > 0.7))
            },
            'clip_score_eeg_captions': {
                'mean': float(np.mean(clip_eeg)),
                'std': float(np.std(clip_eeg)),
                'median': float(np.median(clip_eeg)),
                'min': float(np.min(clip_eeg)),
                'max': float(np.max(clip_eeg))
            },
            'clip_score_gt_captions': {
                'mean': float(np.mean(clip_gt)),
                'std': float(np.std(clip_gt)),
                'median': float(np.median(clip_gt)),
                'min': float(np.min(clip_gt)),
                'max': float(np.max(clip_gt))
            }
        }

        if len(reconstruction) > 0:
            stats['reconstruction_quality'] = {
                'mean': float(np.mean(reconstruction)),
                'std': float(np.std(reconstruction)),
                'median': float(np.median(reconstruction)),
                'min': float(np.min(reconstruction)),
                'max': float(np.max(reconstruction))
            }

        # Compute relative performance
        clip_ratio = stats['clip_score_eeg_captions']['mean'] / stats['clip_score_gt_captions']['mean']
        stats['clip_performance_ratio'] = float(clip_ratio)

        return stats

    def _save_evaluation_examples(self, images, eeg_captions, gt_captions, similarities, save_dir, start_idx):
        """Save visual evaluation examples"""
        for i, (image, eeg_cap, gt_cap, sim) in enumerate(zip(images, eeg_captions, gt_captions, similarities)):
            fig, axes = plt.subplots(1, 3, figsize=(15, 5))

            # Original image
            axes[0].imshow(image)
            axes[0].set_title("Original Image")
            axes[0].axis('off')

            # Generated image from EEG caption
            generated_image = self.generate_image_from_caption(eeg_cap, seed=42 + start_idx + i)
            axes[1].imshow(generated_image)
            axes[1].set_title(f"Generated from EEG\n(Sim: {sim:.3f})")
            axes[1].axis('off')

            # Generated image from GT caption
            gt_generated_image = self.generate_image_from_caption(gt_cap, seed=100 + start_idx + i)
            axes[2].imshow(gt_generated_image)
            axes[2].set_title("Generated from GT")
            axes[2].axis('off')

            # Add captions as text
            fig.suptitle(f"EEG: {eeg_cap}\nGT: {gt_cap}", fontsize=10, y=0.02, wrap=True)

            plt.tight_layout()
            plt.savefig(save_dir / f'example_{start_idx + i}.png', dpi=150, bbox_inches='tight')
            plt.close()

    def print_evaluation_report(self, results_summary):
        """Print a comprehensive evaluation report"""
        print("\n" + "="*80)
        print("🔬 COMPREHENSIVE EVALUATION REPORT")
        print("="*80)

        sem_sim = results_summary['semantic_similarity']
        print(f"\n📝 SEMANTIC SIMILARITY (EEG vs Ground Truth Captions)")
        print(f"   Mean: {sem_sim['mean']:.4f} ± {sem_sim['std']:.4f}")
        print(f"   Median: {sem_sim['median']:.4f}")
        print(f"   Range: [{sem_sim['min']:.4f}, {sem_sim['max']:.4f}]")
        print(f"   High Quality Ratio (>0.7): {sem_sim['high_quality_ratio']:.2%}")

        clip_eeg = results_summary['clip_score_eeg_captions']
        clip_gt = results_summary['clip_score_gt_captions']
        print(f"\n🖼️  CLIP SCORES (Image-Text Alignment)")
        print(f"   EEG Captions:  {clip_eeg['mean']:.4f} ± {clip_eeg['std']:.4f}")
        print(f"   GT Captions:   {clip_gt['mean']:.4f} ± {clip_gt['std']:.4f}")
        print(f"   Relative Performance: {results_summary['clip_performance_ratio']:.2%}")

        if 'reconstruction_quality' in results_summary:
            recon = results_summary['reconstruction_quality']
            print(f"\n🎨 IMAGE RECONSTRUCTION QUALITY")
            print(f"   Mean: {recon['mean']:.4f} ± {recon['std']:.4f}")
            print(f"   Median: {recon['median']:.4f}")
            print(f"   Range: [{recon['min']:.4f}, {recon['max']:.4f}]")

        # Performance assessment
        print(f"\n🏆 OVERALL ASSESSMENT")
        if sem_sim['mean'] > 0.7:
            print("   Semantic Alignment: EXCELLENT ✅")
        elif sem_sim['mean'] > 0.5:
            print("   Semantic Alignment: GOOD 👍")
        else:
            print("   Semantic Alignment: NEEDS IMPROVEMENT 🔄")

        if results_summary['clip_performance_ratio'] > 0.8:
            print("   Image-Text Alignment: EXCELLENT ✅")
        elif results_summary['clip_performance_ratio'] > 0.6:
            print("   Image-Text Alignment: GOOD 👍")
        else:
            print("   Image-Text Alignment: NEEDS IMPROVEMENT 🔄")

        print("="*80)

# ==============================================================================
# --- EVALUATION SCRIPT USAGE ---
# ==============================================================================
def run_evaluation(model_path, config, test_data_path, output_dir=None, num_samples=200):
    """Main evaluation function"""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Initialize evaluator
    evaluator = ModelEvaluator(model_path, config, device)

    # Prepare test data
    spec_mean = torch.load(Path(config.PROCESSED_DATA_ROOT) / 'spec_mean.pt')
    spec_std = torch.load(Path(config.PROCESSED_DATA_ROOT) / 'spec_std.pt')

    transform = transforms.Compose([
        transforms.ToDtype(torch.float32),
        transforms.Resize((224, 224), antialias=True),
        transforms.Normalize(mean=spec_mean.tolist(), std=spec_std.tolist()),
    ])

    # Create test dataset
    test_dataset = EEGDatasetWithCaptions(
        config.PROCESSED_DATA_ROOT,
        config.METADATA_CSV,
        'test',  # or 'val' if no test split
        transform,
        evaluator.processor,
        evaluator.caption_blip,
        device
    )

    # Create test loader
    test_indices = np.random.choice(len(test_dataset), min(len(test_dataset), num_samples), replace=False)
    test_loader = DataLoader(
        test_dataset,
        batch_size=8,
        sampler=SubsetRandomSampler(test_indices),
        num_workers=2
    )

    # Run evaluation
    print(f"🚀 Starting evaluation on {len(test_indices)} samples...")
    results_summary, detailed_results = evaluator.comprehensive_evaluation(
        test_loader,
        num_samples=num_samples,
        save_dir=output_dir
    )

    # Print report
    evaluator.print_evaluation_report(results_summary)

    return evaluator, results_summary, detailed_results

# Example usage
if __name__ == "__main__":
    from complete_training_pipeline import ENHANCED_TRAIN_CONFIG

    # Configuration
    config = ENHANCED_TRAIN_CONFIG()
    model_path = "/path/to/your/best_model.pth"  # Update this path
    output_dir = "/path/to/evaluation/results"   # Update this path

    # Run evaluation
    evaluator, results, detailed = run_evaluation(
        model_path=model_path,
        config=config,
        test_data_path=config.PROCESSED_DATA_ROOT,
        output_dir=output_dir,
        num_samples=100
    )

ModuleNotFoundError: No module named 'complete_training_pipeline'

In [ ]:
import torch
from diffusers import StableDiffusionPipeline
from PIL import Image

# --- 1. Set up the model and pipeline ---
# This will download the model from the Hugging Face Hub the first time you run it.
model_id = "runwayml/stable-diffusion-v1-5"
# Use torch.float16 for faster inference and less memory on NVIDIA GPUs.
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)

# Move the pipeline to the GPU for much faster generation.
# If you don't have a GPU, you can remove this line, but it will be very slow.
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = pipe.to(device)
print(f"Using device: {device}")

# --- 2. Define your text prompt ---
# This is where you'll put the caption you generated.
prompt = "the small brown dog next to the chair is a dog house a small dog kenn"

# --- 3. (Optional) Set a seed for reproducibility ---
# Using a seed ensures you get the same image every time for the same prompt.
generator = torch.Generator(device=device).manual_seed(42)

# --- 4. Generate the image ---
# The pipeline returns an object with the generated image(s).
print("Generating image...")
output = pipe(prompt=prompt, generator=generator)
image = output.images[0] # The generated image is a PIL Image object

# --- 5. Save the image ---
output_path = "generated_image.png"
image.save(output_path)
print(f"Image saved successfully to {output_path}")

# You can also display the image if you are in a Jupyter Notebook or similar environment.
# display(image)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Using device: cuda
Generating image...


  0%|          | 0/50 [00:00<?, ?it/s]

Image saved successfully to generated_image.png


# Old scripts

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
from pathlib import Path
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torchvision.transforms import v2 as transforms
from PIL import Image
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.auto import tqdm
from datetime import datetime
import numpy as np
from transformers import Blip2Processor, Blip2ForConditionalGeneration, Blip2VisionModel
import warnings

# ==============================================================================
# --- 1. CONFIGURATION ---
# ==============================================================================
class TRAIN_CONFIG:
    PROCESSED_DATA_ROOT = '/content/final_lightweight_17k'
    METADATA_CSV = Path(PROCESSED_DATA_ROOT) / 'metadata.csv'
    OUTPUT_DIR = '/content/drive/MyDrive/NeuroVision/EEG_BLIP2_Alignment'

    BLIP_MODEL_NAME = "Salesforce/blip2-opt-2.7b"

    BATCH_SIZE = 2
    GRAD_ACCUMULATION_STEPS = 16
    NUM_EPOCHS = 50
    LR = 5e-5

    WEIGHT_DECAY = 5e-3
    GRADIENT_CLIP_NORM = 1.0

    VALIDATION_INTERVAL = 1
    VISUALIZATION_INTERVAL = 1 # Note: Visualization happens with validation
    VAL_SAMPLES_PER_EPOCH = 700
    TRAIN_SAMPLES_PER_EPOCH = 3500
    VIS_GRID_SIZE = 4

    ALPHA_ALIGN = 1.0      # weight for cosine alignment
    BETA_CE     = 5.0      # weight for cross-entropy (strong)
    GAMMA_EOS   = 10.0

# ==============================================================================
# --- 2. EEG ENCODER MODULE ---
# ==============================================================================

class ChannelAdapter(nn.Module):
    """Lightweight adapter to turn 64-channel spectrogram -> 3-channel-like input for BLIP vision."""
    def __init__(self, in_chans=64, mid_ch=64, out_ch=3):
        super().__init__()
        self.conv1 = nn.Conv2d(in_chans, mid_ch, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(mid_ch)
        self.act = nn.GELU()
        self.conv2 = nn.Conv2d(mid_ch, out_ch, kernel_size=3, padding=1, bias=True)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.act(x)
        x = self.conv2(x)
        return x

class EEGEncoder(nn.Module):
    def __init__(self, config, in_chans=64):
        super().__init__()
        self.vision_model = Blip2VisionModel.from_pretrained(config.BLIP_MODEL_NAME)

        # Freeze the original vision model
        for param in self.vision_model.parameters():
            param.requires_grad = False

        self.adapter = ChannelAdapter(in_chans=in_chans, mid_ch=64, out_ch=3)

        for p in self.adapter.parameters():
            p.requires_grad = True


    def forward(self, x):
      adapted_x = self.adapter(x)
      vision_outputs = self.vision_model(adapted_x, return_dict=True)
      return vision_outputs.last_hidden_state

# ==============================================================================
# --- 3. THE HYBRID EEG-BLIP2 MODEL ---
# ==============================================================================
class EEG_BLIP2_Model(nn.Module):
    def __init__(self, config, device):
        super().__init__()
        self.device = device
        self.eeg_encoder = EEGEncoder(config)
        self.blip = Blip2ForConditionalGeneration.from_pretrained(
            config.BLIP_MODEL_NAME
        )

        # Freeze the entire BLIP model initially
        for param in self.blip.parameters():
            param.requires_grad = False

        print("Unfreezing specified layers for fine-tuning...")
        # Unfreeze the language projection layer (connects Q-Former to LLM)
        if hasattr(self.blip, "language_projection"):
            for p in self.blip.language_projection.parameters():
                p.requires_grad = True

            print("  - Unfroze Language Projection layer.")

        # Unfreeze the last layer(s) of the Q-Former for fine-tuning
        try:
            qenc_layers = self.blip.qformer.encoder.layer
            n_unfreeze_qformer = 2
            for idx in range(len(qenc_layers) - n_unfreeze_qformer, len(qenc_layers)):
                for p in qenc_layers[idx].parameters():
                    p.requires_grad = True
        except Exception:
            pass

        try:
            vision_layers = self.blip.vision_model.encoder.layers
            for p in vision_layers[-2].parameters():
                p.requires_grad = True
            print("  - Unfroze the last Vision Model layer.")
        except Exception as e:
            print(f"Could not unfreeze Vision Model layer: {e}")

        # 4. Unfreeze the last Language Model layer
        try:
            # Note: The exact path might differ slightly between transformers versions
            llm_layers = self.blip.language_model.model.decoder.layers
            for p in llm_layers[-2].parameters():
                p.requires_grad = True
            print("  - Unfroze the last Language Model layer.")
        except Exception as e:
            print(f"Could not unfreeze Language Model layer: {e}")


        self.eeg_encoder.to(device)
        self.blip.to(device)

    def get_eeg_embedding(self, eeg_spectrograms):
        eeg_features = self.eeg_encoder(eeg_spectrograms)
        query_tokens = self.blip.query_tokens.expand(eeg_features.shape[0], -1, -1)
        query_outputs = self.blip.qformer(
            query_embeds=query_tokens,
            encoder_hidden_states=eeg_features,
            return_dict=True,
        )
        return query_outputs.last_hidden_state.mean(dim=1)

    def get_image_embedding(self, pil_images, processor):
        inputs = processor(images=pil_images, return_tensors="pt").to(self.device)
        pixel_values = inputs.pixel_values

        image_features = self.blip.vision_model(pixel_values).last_hidden_state
        query_tokens = self.blip.query_tokens.expand(image_features.shape[0], -1, -1)
        query_outputs = self.blip.qformer(
            query_embeds=query_tokens,
            encoder_hidden_states=image_features,
            return_dict=True,
        )
        return query_outputs.last_hidden_state.mean(dim=1)

    def generate(self, eeg_spectrograms, **kwargs):
        with torch.no_grad():
            eeg_features = self.eeg_encoder(eeg_spectrograms)
            query_tokens = self.blip.query_tokens.expand(eeg_features.shape[0], -1, -1)
            query_outputs = self.blip.qformer(
                query_embeds=query_tokens,
                encoder_hidden_states=eeg_features,
                return_dict=True,
            )
            language_model_inputs = self.blip.language_projection(query_outputs.last_hidden_state)
            generated_ids = self.blip.language_model.generate(
                inputs_embeds=language_model_inputs,
                **kwargs
            )
            return generated_ids

# ==============================================================================
# --- 4. DATASET & COLLATING ---
# ==============================================================================
def collate_fn(batch):
    spectrograms = torch.stack([item[0] for item in batch])
    pil_images = [item[1] for item in batch]
    return spectrograms, pil_images

class EEGDataset(Dataset):
    def __init__(self, root_dir, metadata_csv, split, transform):
        self.root_dir = Path(root_dir)
        self.transform = transform
        df = pd.read_csv(metadata_csv)
        self.split_df = df[df['split'].str.strip() == split].reset_index(drop=True)

    def __len__(self): return len(self.split_df)

    def __getitem__(self, idx):
        info = self.split_df.iloc[idx]
        spectrogram = self.transform(torch.load(self.root_dir / info['spectrogram_path']))
        image = Image.open(self.root_dir / info['image_path']).convert("RGB")
        return spectrogram, image

# ==============================================================================
# --- 5. MAIN TRAINING FUNCTION ---
# ==============================================================================
# InfoNCE loss function
def info_nce_loss(query, positive_key, negative_keys=None, temperature=0.07):
    # Ensure inputs are normalized
    query = nn.functional.normalize(query, dim=-1)
    positive_key = nn.functional.normalize(positive_key, dim=-1)

    # Calculate positive logits
    positive_logits = torch.sum(query * positive_key, dim=-1, keepdim=True)

    # Concatenate all other items in the batch as negative keys
    if negative_keys is None:
        negative_keys = positive_key

    negative_logits = query @ negative_keys.T

    # Mask out self-comparison for the InfoNCE formula
    logits = torch.cat([positive_logits, negative_logits], dim=1)

    # Create labels for cross-entropy: first element is the positive pair
    labels = torch.zeros(len(query), dtype=torch.long, device=query.device)

    return nn.functional.cross_entropy(logits / temperature, labels)

#------------------------------------------------------------------------------------------
def train_model(config):
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    output_dir = Path(config.OUTPUT_DIR) / f'run_{timestamp}'
    output_dir.mkdir(parents=True, exist_ok=True)

    print("🚀 Starting EEG-to-Text Training with Embedding Alignment Loss")

    processor = Blip2Processor.from_pretrained(config.BLIP_MODEL_NAME)
    student_model = EEG_BLIP2_Model(config, device)

    spec_mean = torch.load(Path(config.PROCESSED_DATA_ROOT) / 'spec_mean.pt')
    spec_std = torch.load(Path(config.PROCESSED_DATA_ROOT) / 'spec_std.pt')
    transform = transforms.Compose([
        transforms.ToDtype(torch.float32),
        transforms.Resize((224, 224), antialias=True),
        transforms.Normalize(mean=spec_mean.tolist(), std=spec_std.tolist()),
    ])
    train_dataset = EEGDataset(config.PROCESSED_DATA_ROOT, config.METADATA_CSV, 'train', transform)
    val_dataset = EEGDataset(config.PROCESSED_DATA_ROOT, config.METADATA_CSV, 'val', transform)

    val_indices = np.random.choice(len(val_dataset), min(len(val_dataset), config.VAL_SAMPLES_PER_EPOCH), replace=False)
    val_loader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE, sampler=SubsetRandomSampler(val_indices), collate_fn=collate_fn)

    fixed_spectrograms, fixed_pil_images = next(iter(val_loader))
    fixed_spectrograms = fixed_spectrograms.to(device)

    # OPTIMIZED: Only pass trainable parameters to the optimizer
    trainable_params = filter(lambda p: p.requires_grad, student_model.parameters())
    optimizer = optim.AdamW(trainable_params, lr=config.LR, weight_decay=config.WEIGHT_DECAY)

    # scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1, eta_min=1e-7)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=2, min_lr=1e-7)
    scaler = torch.cuda.amp.GradScaler()
    loss_fn = nn.CosineEmbeddingLoss()
    best_val_loss = float('inf')

    print("🎯 Starting Training Loop")
    for epoch in range(config.NUM_EPOCHS):
        student_model.train()
        train_indices = np.random.choice(len(train_dataset), min(len(train_dataset), config.TRAIN_SAMPLES_PER_EPOCH), replace=False)
        train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, sampler=SubsetRandomSampler(train_indices), drop_last=True, collate_fn=collate_fn)
        train_bar = tqdm(train_loader, desc=f"🔄 Epoch {epoch+1}/{config.NUM_EPOCHS} [TRAIN]")

        optimizer.zero_grad()
        for batch_idx, (spectrograms, pil_images) in enumerate(train_bar):
            spectrograms = spectrograms.to(device)

            with torch.amp.autocast('cuda'):
                eeg_embedding = student_model.get_eeg_embedding(spectrograms)

                with torch.no_grad():
                    image_embedding = student_model.get_image_embedding(pil_images, processor)

                target = torch.ones(eeg_embedding.size(0)).to(device)
                loss_align = info_nce_loss(eeg_embedding, image_embedding, temperature=0.07)
                cse_loss = loss_fn(eeg_embedding, image_embedding, target)

                # CORRECTED: Was `loss = loss / ...` but `loss` wasn't defined. Use `loss_align`.
                loss = (2.0 * loss_align + cse_loss) / config.GRAD_ACCUMULATION_STEPS

            scaler.scale(loss).backward()

            if (batch_idx + 1) % config.GRAD_ACCUMULATION_STEPS == 0:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(student_model.parameters(), config.GRADIENT_CLIP_NORM)
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()

            # IMPROVED: Show the actual alignment loss, not the scaled version
            train_bar.set_postfix(align_loss=f"{loss_align.item():.4f}")

        if (epoch + 1) % config.VALIDATION_INTERVAL == 0:
            print(f"\n🔍 Running Validation & Visualization for epoch {epoch+1}")
            student_model.eval()
            val_loss = 0

            with torch.no_grad(), torch.amp.autocast('cuda'):
                for spectrograms, pil_images in val_loader:
                    spectrograms = spectrograms.to(device)
                    eeg_embedding = student_model.get_eeg_embedding(spectrograms)
                    image_embedding = student_model.get_image_embedding(pil_images, processor)
                    target = torch.ones(eeg_embedding.size(0)).to(device)
                    cse_loss = loss_fn(eeg_embedding, image_embedding, target).item()
                    loss_align = info_nce_loss(eeg_embedding, image_embedding, temperature=0.07)
                    loss = 2.0 * loss_align + cse_loss
                    val_loss += loss

            avg_val_loss = val_loss / len(val_loader)
            print(f"  Validation Loss: {avg_val_loss:.4f}")
            scheduler.step(avg_val_loss)

            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                # Save only the trainable part of the model: the EEG encoder and fine-tuned layers
                torch.save(student_model.state_dict(), output_dir / 'best_model.pth')
                print(f"🏆 New best model saved with validation loss: {best_val_loss:.4f}")

            with torch.no_grad(), torch.amp.autocast('cuda'):
                generated_ids = student_model.generate(
                    fixed_spectrograms[:config.VIS_GRID_SIZE],
                    do_sample=True, temperature=1.0, top_p=0.9,
                    min_new_tokens=8, max_new_tokens=32, repetition_penalty=1.5
                    # num_beams=4
                )
                predicted_captions = processor.batch_decode(generated_ids, skip_special_tokens=True)

                # Get ground truth captions
                # CORRECTED: Removed explicit .to(dtype=torch.float16) for robustness
                gt_pixel_values = processor(images=fixed_pil_images, return_tensors="pt").to(device).pixel_values
                gt_captions_ids = student_model.blip.generate(gt_pixel_values)
                gt_captions = processor.batch_decode(gt_captions_ids, skip_special_tokens=True)


                print("\n--- Caption Generation Samples ---")
                for i in range(len(predicted_captions)):
                    print(f"Sample {i+1}:")
                    print(f"  - Ground Truth: {gt_captions[i].strip()}")
                    print(f"  - Predicted:    {predicted_captions[i].strip()}")
                print("------------------------------------")

    print("\n🎉 Training Complete!")
    return output_dir

if __name__ == '__main__':
    config = TRAIN_CONFIG()
    try:
        output_dir = train_model(config)
        if output_dir:
            print(f"✅ Training completed successfully! Results saved to: {output_dir}")
    except Exception as e:
        print(f"❌ Training failed with error: {str(e)}")
        import traceback
        traceback.print_exc()

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


🚀 Starting EEG-to-Text Training with Embedding Alignment Loss


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

/tmp/ipython-input-779291635.py:263: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


🔄 Epoch 1/50 [TRAIN]:   0%|          | 0/1750 [00:00<?, ?it/s]


🔍 Running Validation & Visualization for epoch 1
  Validation Loss: 2.2821
🏆 New best model saved with validation loss: 2.2821

--- Caption Generation Samples ---
Sample 1:
  - Ground Truth: a man with a donut in his mouth
  - Predicted:    david taylor in the desert. all rights reserved 2012 dalton brown
Sample 2:
  - Ground Truth: a pair of scissors
  - Predicted:    color of the day with a blue door
------------------------------------


🔄 Epoch 2/50 [TRAIN]:   0%|          | 0/1750 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Dump

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from pathlib import Path
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torchvision.transforms import v2 as transforms
from PIL import Image
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.auto import tqdm
from datetime import datetime
import numpy as np
import json
import clip
from sentence_transformers import SentenceTransformer
from diffusers import StableDiffusionPipeline
from transformers import Blip2Processor, Blip2ForConditionalGeneration, Blip2VisionModel

# ==============================================================================
# --- 1. CONFIGURATION (MERGED) ---
# ==============================================================================
class ENHANCED_CONFIG:
    # --- Paths and Models ---
    PROCESSED_DATA_ROOT = '/content/final_lightweight_17k'
    METADATA_CSV = Path(PROCESSED_DATA_ROOT) / 'metadata.csv'
    OUTPUT_DIR = '/content/drive/MyDrive/NeuroVision/EEG_BLIP2_Alignment_Stable'
    BLIP_MODEL_NAME = "Salesforce/blip2-opt-2.7b"
    CLIP_MODEL_NAME = "ViT-B/32"
    DIFFUSION_MODEL_ID = "runwayml/stable-diffusion-v1-5"

    # --- Stable Hyperparameters ---
    BATCH_SIZE = 32
    LR = 1e-5
    WEIGHT_DECAY = 5e-3
    GRADIENT_CLIP_NORM = 1.0
    GRAD_ACCUMULATION_STEPS = 1

    # --- Training Loop Settings ---
    NUM_EPOCHS = 50
    TRAIN_SAMPLES_PER_EPOCH = 3500
    VAL_SAMPLES_PER_EPOCH = 700

    # --- New Loss Weights ---
    EMBEDDING_WEIGHT = 1.0
    CAPTION_SIM_WEIGHT = 3.0
    CLIP_ALIGNMENT_WEIGHT = 0.25

    # --- Evaluation Settings ---
    EVAL_IMAGE_GENERATION_INTERVAL = 1
    EVAL_SAMPLES_TO_GENERATE = 3 # How many images to generate each evaluation

# ==============================================================================
# --- 2. STABLE EEG ENCODER MODULE (FROM OLD SCRIPT) ---
# ==============================================================================
class ChannelAdapter(nn.Module):
    def __init__(self, in_chans=64, mid_ch=64, out_ch=3):
        super().__init__()
        self.conv1 = nn.Conv2d(in_chans, mid_ch, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(mid_ch)
        self.act = nn.GELU()
        self.conv2 = nn.Conv2d(mid_ch, out_ch, kernel_size=3, padding=1, bias=True)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.act(x)
        x = self.conv2(x)
        return x

class EEGEncoder(nn.Module):
    """
    A simplified, stable encoder that ONLY contains the adapter.
    It receives the vision model during its forward pass.
    """
    def __init__(self, in_chans=64):
        super().__init__()
        self.adapter = ChannelAdapter(in_chans=in_chans)

    def forward(self, x, vision_model):
      # Takes the main vision model as an argument
      adapted_x = self.adapter(x)
      vision_outputs = vision_model(adapted_x, return_dict=True)
      return vision_outputs.last_hidden_state

# ==============================================================================
# --- 3. STABLE HYBRID EEG-BLIP2 MODEL (FROM OLD SCRIPT) ---
# ==============================================================================
class EEG_BLIP2_Model(nn.Module):
    def __init__(self, config, device):
        super().__init__()
        self.device = device
        self.eeg_encoder = EEGEncoder() # Now much simpler
        self.blip = Blip2ForConditionalGeneration.from_pretrained(config.BLIP_MODEL_NAME)

        # Freeze the entire BLIP model initially
        for param in self.blip.parameters():
            param.requires_grad = False

        print("Unfreezing specified layers for fine-tuning...")
        # Unfreeze the language projection layer (connects Q-Former to LLM)
        if hasattr(self.blip, "language_projection"):
            for p in self.blip.language_projection.parameters():
                p.requires_grad = True

            print("  - Unfroze Language Projection layer.")

        # Unfreeze the last layer(s) of the Q-Former for fine-tuning
        try:
            qenc_layers = self.blip.qformer.encoder.layer
            n_unfreeze_qformer = 4
            for idx in range(len(qenc_layers) - n_unfreeze_qformer, len(qenc_layers)):
                for p in qenc_layers[idx].parameters():
                    p.requires_grad = True
        except Exception:
            pass

        try:
            vision_layers = self.blip.vision_model.encoder.layers
            for p in vision_layers[-4].parameters():
                p.requires_grad = True
            print("  - Unfroze the last Vision Model layer.")
        except Exception as e:
            print(f"Could not unfreeze Vision Model layer: {e}")

        # 4. Unfreeze the last Language Model layer
        try:
            # Note: The exact path might differ slightly between transformers versions
            llm_layers = self.blip.language_model.model.decoder.layers
            for p in llm_layers[-4].parameters():
                p.requires_grad = True
            print("  - Unfroze the last Language Model layer.")
        except Exception as e:
            print(f"Could not unfreeze Language Model layer: {e}")

        self.eeg_encoder.to(device)
        self.blip.to(device)

    def get_eeg_embedding(self, eeg_spectrograms):
        # Pass the single, authoritative vision model to the encoder
        eeg_features = self.eeg_encoder(eeg_spectrograms, self.blip.vision_model)

        query_tokens = self.blip.query_tokens.expand(eeg_features.shape[0], -1, -1)
        query_outputs = self.blip.qformer(
            query_embeds=query_tokens, encoder_hidden_states=eeg_features, return_dict=True)
        return query_outputs.last_hidden_state.mean(dim=1)

    def get_image_embedding(self, pil_images, processor):
        inputs = processor(images=pil_images, return_tensors="pt").to(self.device)
        pixel_values = inputs.pixel_values

        # Use the single, authoritative vision model
        image_features = self.blip.vision_model(pixel_values).last_hidden_state
        query_tokens = self.blip.query_tokens.expand(image_features.shape[0], -1, -1)
        query_outputs = self.blip.qformer(
            query_embeds=query_tokens, encoder_hidden_states=image_features, return_dict=True)
        return query_outputs.last_hidden_state.mean(dim=1)

    def generate(self, eeg_spectrograms, **kwargs):
        with torch.no_grad():
            # Pass the single, authoritative vision model to the encoder
            eeg_features = self.eeg_encoder(eeg_spectrograms, self.blip.vision_model)

            query_tokens = self.blip.query_tokens.expand(eeg_features.shape[0], -1, -1)
            query_outputs = self.blip.qformer(
                query_embeds=query_tokens, encoder_hidden_states=eeg_features, return_dict=True)
            language_model_inputs = self.blip.language_projection(query_outputs.last_hidden_state)
            generated_ids = self.blip.language_model.generate(
                inputs_embeds=language_model_inputs, **kwargs)
            return generated_ids

# ==============================================================================
# --- 4. DATASET & HELPERS (MERGED) ---
# ==============================================================================
class EEGDatasetWithCaptions(Dataset):
    def __init__(self, root_dir, metadata_csv, split, transform):
        self.root_dir = Path(root_dir)
        self.transform = transform
        df = pd.read_csv(metadata_csv)
        self.split_df = df[df['split'].str.strip() == split].reset_index(drop=True)

        captions_path = Path(root_dir) / f'{split}_gt_captions.json'
        if not captions_path.exists():
            raise FileNotFoundError(f"Caption file not found: {captions_path}. Please run the caption generation script first.")
        print(f"Loading pre-generated captions from {captions_path}")
        with open(captions_path, 'r') as f:
            self.captions = json.load(f)

    def __len__(self):
        return len(self.split_df)

    def __getitem__(self, idx):
        info = self.split_df.iloc[idx]
        spectrogram = self.transform(torch.load(self.root_dir / info['spectrogram_path']))
        image = Image.open(self.root_dir / info['image_path']).convert("RGB")
        gt_caption = self.captions.get(str(idx), "an image")
        return spectrogram, image, gt_caption

def collate_fn(batch):
    spectrograms = torch.stack([item[0] for item in batch])
    pil_images = [item[1] for item in batch]
    captions = [item[2] for item in batch]
    return spectrograms, pil_images, captions

# ==============================================================================
# --- 5. ENHANCED LOSS FUNCTIONS (FROM NEW SCRIPT) ---
# ==============================================================================
class CLIPAlignmentLoss(nn.Module):
    def __init__(self, device, model_name="ViT-B/32"):
        super().__init__()
        self.device = device
        self.clip_model, self.clip_preprocess = clip.load(model_name, device=device)
        for param in self.clip_model.parameters():
            param.requires_grad = False

    def forward(self, images, texts):
        if isinstance(images[0], Image.Image):
            image_inputs = torch.stack([self.clip_preprocess(img) for img in images]).to(self.device)
        else:
            image_inputs = images.to(self.device)
        text_inputs = clip.tokenize(texts, truncate=True).to(self.device)
        with torch.no_grad():
            image_features = self.clip_model.encode_image(image_inputs)
            text_features = self.clip_model.encode_text(text_inputs)
        image_features = F.normalize(image_features, dim=-1)
        text_features = F.normalize(text_features, dim=-1)
        similarity = torch.sum(image_features * text_features, dim=-1)
        return -similarity.mean()

class ComprehensiveLoss(nn.Module):
    def __init__(self, device, config):
        super().__init__()
        self.device = device
        self.config = config
        self.sentence_model = SentenceTransformer('all-MiniLM-L6-v2').to(device)
        self.clip_loss = CLIPAlignmentLoss(device, config.CLIP_MODEL_NAME)
        self.cosine_loss = nn.CosineEmbeddingLoss()
        for param in self.sentence_model.parameters():
            param.requires_grad = False

    def info_nce_loss(self, query, positive_key, temperature=0.07):
        query = F.normalize(query, dim=-1)
        positive_key = F.normalize(positive_key, dim=-1)
        positive_logits = torch.sum(query * positive_key, dim=-1, keepdim=True)
        negative_logits = query @ positive_key.T
        logits = torch.cat([positive_logits, negative_logits], dim=1) / temperature
        labels = torch.zeros(len(query), dtype=torch.long, device=self.device)
        return F.cross_entropy(logits, labels)

    def forward(self, eeg_embeddings, image_embeddings, eeg_captions, gt_captions, pil_images):
        losses = {}
        eeg_embeddings = F.normalize(eeg_embeddings, dim=-1)
        image_embeddings = F.normalize(image_embeddings, dim=-1)

        losses['embedding_contrastive'] = self.info_nce_loss(eeg_embeddings, image_embeddings)

        with torch.no_grad():
            eeg_caption_embeddings = self.sentence_model.encode(eeg_captions, convert_to_tensor=True)
            gt_caption_embeddings = self.sentence_model.encode(gt_captions, convert_to_tensor=True)

        caption_sim = F.cosine_similarity(eeg_caption_embeddings, gt_caption_embeddings, dim=-1)
        losses['caption_similarity'] = 1.0 - caption_sim.mean()

        losses['clip_alignment'] = self.clip_loss(pil_images, eeg_captions)

        total_loss = (
            self.config.EMBEDDING_WEIGHT * losses['embedding_contrastive'] +
            self.config.CAPTION_SIM_WEIGHT * losses['caption_similarity'] +
            self.config.CLIP_ALIGNMENT_WEIGHT * losses['clip_alignment']
        )
        losses['total'] = total_loss
        return losses

# ==============================================================================
# --- 6. IMAGE EVALUATION (FROM NEW SCRIPT) ---
# ==============================================================================
class ImageGenerationEvaluator:
    def __init__(self, device, config):
        self.device = device
        self.config = config
        print("Loading Stable Diffusion pipeline...")
        self.diffusion_pipe = StableDiffusionPipeline.from_pretrained(
            config.DIFFUSION_MODEL_ID,
            torch_dtype=torch.float16 if device.type == "cuda" else torch.float32,
            safety_checker=None,
            requires_safety_checker=False
        ).to(device)
        self.diffusion_pipe.set_progress_bar_config(disable=True)
        self.eval_dir = Path(config.OUTPUT_DIR) / 'generated_images'
        self.eval_dir.mkdir(parents=True, exist_ok=True)

    def generate_and_save_samples(self, eeg_captions, original_images, epoch):
        print(f"\n🖼️  Generating {len(eeg_captions)} images for evaluation...")
        for i, (caption, original_image) in enumerate(zip(eeg_captions, original_images)):
            generator = torch.Generator(device=self.device).manual_seed(42 + i)
            generated_image = self.diffusion_pipe(
                prompt=caption, generator=generator, num_inference_steps=20
            ).images[0]

            comparison = Image.new('RGB', (1024, 512))
            comparison.paste(original_image.resize((512, 512)), (0, 0))
            comparison.paste(generated_image.resize((512, 512)), (512, 0))

            comparison_path = self.eval_dir / f'epoch_{epoch}_sample_{i}.png'
            comparison.save(comparison_path)
            print(f"  Saved sample {i+1} to {comparison_path}")

# ==============================================================================
# --- 7. MAIN TRAINING FUNCTION (MERGED AND CORRECTED) ---
# ==============================================================================
def train_model(config):
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    output_dir = Path(config.OUTPUT_DIR) / f'run_{timestamp}'
    output_dir.mkdir(parents=True, exist_ok=True)

    print("🚀 Starting Merged EEG-to-Text Training")

    processor = Blip2Processor.from_pretrained(config.BLIP_MODEL_NAME)
    model = EEG_BLIP2_Model(config, device)

    generation_args = {
    "max_new_tokens": 30,  # Generate up to 30 new words
    "min_length": 8,       # Generate at least 8 words
    "num_beams": 3,        # Use 3 beams for higher quality search
    "do_sample": False     # Use deterministic generation for loss calculation
    }

    comprehensive_loss = ComprehensiveLoss(device, config)
    image_evaluator = ImageGenerationEvaluator(device, config)

    spec_mean = torch.load(Path(config.PROCESSED_DATA_ROOT) / 'spec_mean.pt')
    spec_std = torch.load(Path(config.PROCESSED_DATA_ROOT) / 'spec_std.pt')
    spec_std += 1e-6

    transform = transforms.Compose([
        transforms.ToDtype(torch.float32),
        transforms.Resize((224, 224), antialias=True),
        transforms.Normalize(mean=spec_mean.tolist(), std=spec_std.tolist()),
    ])

    train_dataset = EEGDatasetWithCaptions(config.PROCESSED_DATA_ROOT, config.METADATA_CSV, 'train', transform)
    val_dataset = EEGDatasetWithCaptions(config.PROCESSED_DATA_ROOT, config.METADATA_CSV, 'val', transform)

    trainable_params = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = optim.AdamW(trainable_params, lr=config.LR, weight_decay=config.WEIGHT_DECAY)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=2, min_lr=1e-7)
    scaler = torch.cuda.amp.GradScaler()
    best_val_loss = float('inf')
    simple_alignment_loss = nn.CosineEmbeddingLoss()

    print("🎯 Starting Training Loop")
    for epoch in range(config.NUM_EPOCHS):
        model.train()
        train_indices = np.random.choice(len(train_dataset), min(len(train_dataset), config.TRAIN_SAMPLES_PER_EPOCH), replace=False)
        train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, sampler=SubsetRandomSampler(train_indices), drop_last=True, collate_fn=collate_fn)
        train_bar = tqdm(train_loader, desc=f"🔄 Epoch {epoch+1}/{config.NUM_EPOCHS} [TRAIN]")

        for batch_idx, (spectrograms, pil_images, gt_captions) in enumerate(train_bar):
            spectrograms = spectrograms.to(device)

            with torch.amp.autocast('cuda'):
                eeg_embeddings = model.get_eeg_embedding(spectrograms)

                with torch.no_grad():
                    image_embeddings = model.get_image_embedding(pil_images, processor)

                if epoch < 10: # STAGE 1: Foundational Alignment
                    target = torch.ones(eeg_embeddings.size(0)).to(device)
                    loss = simple_alignment_loss(eeg_embeddings, image_embeddings, target)
                else: # STAGE 2: Full Comprehensive Loss
                    with torch.no_grad():
                        generated_ids = model.generate(spectrograms, **generation_args)
                        eeg_captions = processor.batch_decode(generated_ids, skip_special_tokens=True)

                    loss_dict = comprehensive_loss(eeg_embeddings, image_embeddings, eeg_captions, gt_captions, pil_images)
                    loss = loss_dict['total'] / config.GRAD_ACCUMULATION_STEPS

            scaler.scale(loss).backward()

            if (batch_idx + 1) % config.GRAD_ACCUMULATION_STEPS == 0:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(
                    filter(lambda p: p.requires_grad, model.parameters()),
                    config.GRADIENT_CLIP_NORM
                )
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()

            train_bar.set_postfix(loss=loss.item() * config.GRAD_ACCUMULATION_STEPS)

        # --- Validation Loop ---
        print(f"\n🔍 Running Validation for epoch {epoch+1}")
        model.eval()
        val_losses = []
        eval_samples = [] # To store samples for image generation

        val_indices = np.random.choice(len(val_dataset), min(len(val_dataset), config.VAL_SAMPLES_PER_EPOCH), replace=False)
        val_loader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE, sampler=SubsetRandomSampler(val_indices), collate_fn=collate_fn)

        with torch.no_grad(), torch.amp.autocast('cuda'):
            for spectrograms, pil_images, gt_captions in val_loader:
                spectrograms = spectrograms.to(device)
                eeg_embeddings = model.get_eeg_embedding(spectrograms)
                image_embeddings = model.get_image_embedding(pil_images, processor)
                generated_ids = model.generate(spectrograms, **generation_args)
                eeg_captions = processor.batch_decode(generated_ids, skip_special_tokens=True)

                # loss_dict = comprehensive_loss(
                #     eeg_embeddings, image_embeddings, eeg_captions, gt_captions, pil_images)
                # val_losses.append(loss_dict['total'].item())
                target = torch.ones(eeg_embeddings.size(0)).to(device)
                val_losses.append(simple_alignment_loss(eeg_embeddings, image_embeddings, target).item())

                # Collect samples for image generation
                if len(eval_samples) < config.EVAL_SAMPLES_TO_GENERATE:
                    for i in range(len(eeg_captions)):
                        if len(eval_samples) < config.EVAL_SAMPLES_TO_GENERATE:
                            eval_samples.append((eeg_captions[i], gt_captions[i], pil_images[i]))

        avg_val_loss = np.mean(val_losses)
        print(f"  Validation Loss: {avg_val_loss:.4f}")
        scheduler.step(avg_val_loss)

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), output_dir / 'best_model.pth')
            print(f"🏆 New best model saved with validation loss: {best_val_loss:.4f}")

        if eval_samples:
            print("\n--- ✍️  Caption Generation Samples ---")
            for i, (predicted_cap, gt_cap, _) in enumerate(eval_samples):
              print(f"Sample {i+1}:")
              print(f"  - Ground Truth: {gt_cap.strip()}")
              print(f"  - Predicted:    {predicted_cap.strip()}")
              print("------------------------------------")

        # --- Image Generation Evaluation ---
        if (epoch + 1) % config.EVAL_IMAGE_GENERATION_INTERVAL == 0 and eval_samples:
            eval_captions,  _, eval_images = zip(*eval_samples)
            image_evaluator.generate_and_save_samples(list(eval_captions), list(eval_images), epoch + 1)

    print("\n🎉 Training Complete!")
    return output_dir

if __name__ == '__main__':
    # It's recommended to run caption generation separately first.
    # For now, we assume the JSON files exist.
    config = ENHANCED_CONFIG()
    try:
        output_dir = train_model(config)
        if output_dir:
            print(f"✅ Training completed successfully! Results saved to: {output_dir}")
    except Exception as e:
        print(f"❌ Training failed with error: {str(e)}")
        import traceback
        traceback.print_exc()

In [ ]:
def generate_and_cache_captions(config, processor, blip_model, device):
    """
    A one-time function to generate and save captions for train and val splits.
    """
    for split in ['train', 'val']:
        captions_path = Path(config.PROCESSED_DATA_ROOT) / f'{split}_gt_captions.json'
        if captions_path.exists():
            print(f"Captions for '{split}' split already exist at {captions_path}. Skipping.")
            continue

        print(f"Generating ground truth captions for '{split}' split...")
        df = pd.read_csv(config.METADATA_CSV)
        split_df = df[df['split'].str.strip() == split].reset_index(drop=True)

        captions = {}
        batch_size = 16
        blip_model.eval()

        with torch.no_grad():
            for i in tqdm(range(0, len(split_df), batch_size), desc=f"Generating {split} captions"):
                batch_info = split_df.iloc[i:i+batch_size]
                batch_images = [
                    Image.open(Path(config.PROCESSED_DATA_ROOT) / info['image_path']).convert("RGB")
                    for _, info in batch_info.iterrows()
                ]

                inputs = processor(images=batch_images, return_tensors="pt").to(device)
                generated_ids = blip_model.generate(**inputs, max_length=50)
                batch_captions = processor.batch_decode(generated_ids, skip_special_tokens=True)

                for j, caption in enumerate(batch_captions):
                    captions[str(i + j)] = caption.strip()

        with open(captions_path, 'w') as f:
            json.dump(captions, f, indent=2)
        print(f"✅ Saved {len(captions)} captions for '{split}' split to {captions_path}")
